In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import math
import warnings


warnings.filterwarnings('ignore')

print("All libraries imported successfully.")


All libraries imported successfully.


In [7]:
# --- Load the Dataset ---
# We use the cleaned CSV file provided.
file_path = '/Users/clarencemarvin/Downloads/centanet_cleaned_proximity.csv'

try:
    data = pd.read_csv(file_path)
    print(f"Dataset loaded successfully: {file_path}")
    print(f"Dataset shape: {data.shape}")
except FileNotFoundError:
    print(f"ERROR: File not found at {file_path}")
    print("Please make sure the file 'centanet_data_cleaned_for_model.csv' is in the correct folder.")

# Display basic data info to confirm
if 'data' in locals():
    print("\nData Head:")
    print(data.head())
    print("\nData Types:")
    print(data.info())

Dataset loaded successfully: /Users/clarencemarvin/Downloads/centanet_cleaned_proximity.csv
Dataset shape: (6156, 22)

Data Head:
                           property_name            district  bedroom_count  \
0                    LAKE SILVER Tower 5          Wu Kai Sha              2   
1      TSUEN KING GARDEN PHASE 2 BLOCK 8  Tsuen King Circuit              3   
2       BELVEDERE GARDEN PHASE 1 BLOCK 3           Belvedere              3   
3             MARBELLA BLOCK 1 (TOWER A)          Ma On Shan              3   
4  LOHAS PARK PHASE 10 LP10 Tower 1 (1A)          Lohas Park              3   

      price  property_age  saleable_area  pet_policy   latitude   longitude  \
0   7600000            16            573        True  22.428921  114.244390   
1   5800000            37            577       False  22.378113  114.110063   
2   5400000            38            644       False  22.372633  114.101914   
3   7300000            24            737        True  22.427676  114.240275   


In [8]:
# Dataset 1: Manual encoding for RF and SVR
print("=== Creating Dataset 1: Manual Encoding (for RF and SVR) ===")

# Create a copy for manual encoding
data_encoded = data.copy()

# One-hot encode categorical features
categorical_features = ['property_name', 'district']
data_encoded_dummy = pd.get_dummies(data_encoded, columns=categorical_features, prefix=categorical_features)

print(f"Original dataset shape: {data.shape}")
print(f"After one-hot encoding shape: {data_encoded_dummy.shape}")
print(f"Added {data_encoded_dummy.shape[1] - data.shape[1]} new columns from encoding")

# Prepare X and y for encoded dataset
X_encoded = data_encoded_dummy.drop('price', axis=1)
y_encoded = data_encoded_dummy['price']

print(f"Encoded dataset columns: {list(X_encoded.columns)}")

# ===================================================================

print("\n=== Creating Dataset 2: Native Categorical (for XGB, CatBoost, LightGBM) ===")

# Create a copy for native categorical handling
data_categorical = data.copy()

# Convert categorical columns to proper dtype
data_categorical['property_name'] = data_categorical['property_name'].astype('category')
data_categorical['district'] = data_categorical['district'].astype('category')

print("Categorical columns info:")
print(f"property_name - {data_categorical['property_name'].nunique()} unique categories")
print(f"district - {data_categorical['district'].nunique()} unique categories")

# Prepare X and y for categorical dataset
X_categorical = data_categorical.drop('price', axis=1)
y_categorical = data_categorical['price']

print(f"Categorical dataset shape: {X_categorical.shape}")
print(f"Data types:")
print(X_categorical.dtypes)

print("\n=== Both datasets ready! ===")
print("Dataset 1 (X_encoded, y_encoded) - for RF and SVR")
print("Dataset 2 (X_categorical, y_categorical) - for XGB, CatBoost, LightGBM")

=== Creating Dataset 1: Manual Encoding (for RF and SVR) ===
Original dataset shape: (6156, 22)
After one-hot encoding shape: (6156, 4418)
Added 4396 new columns from encoding
Encoded dataset columns: ['bedroom_count', 'property_age', 'saleable_area', 'pet_policy', 'latitude', 'longitude', 'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m', 'category_Community_Facilities_within_1000m', 'category_Education_within_2000m', 'category_Recreation_within_1000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'category_Religion_within_2000m', 'category_Transportation_within_1000m', 'category_Tourism_within_2000m', 'distance_to_nearest_mtr_km', 'distance_to_nearest_match_km', 'property_name_#LYOS Maisonette Building C', 'property_name_10 LASALLE', 'property_name_128 WATERLOO', 'property_name_128 Waterloo', 'property_name_15-15A HO MAN TIN STREET', 'property_name_18 Upper East', 'property_name_2Gether', 'property_name_3 CHICO TERRACE', 'property_name_3

In [9]:
# --- Feature Engineering and Preprocessing ---

# 1. Drop columns not useful for modeling
columns_to_drop = ['latitude', 'longitude', 'walking_time_to_mtr','category_Community_Facilities_within_1000m', 'distance_to_nearest_match_km', 'distance_to_nearest_mtr_km', 'category_Recreation_within_1000m', 'category_Religion_within_2000m', 'category_Tourism_within_2000m', 'category_Transportation_within_1000m']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

# 2. Feature Engineering: Convert 'pet_policy' to a binary feature
# 1 if a policy is listed (Not-NA), 0 if it's missing (NA)
if 'pet_policy' in data.columns:
    data['pet_policy_binary'] = data['pet_policy'].notna().astype(int)
    data = data.drop('pet_policy', axis=1)
    print("Feature 'pet_policy_binary' created.")

# 3. Define the target variable (y) and features (X)
y = data['price']
X = data.drop('price', axis=1)

# 4. Identify numerical and categorical columns
# We will need these lists for our preprocessors
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nIdentified {len(numerical_cols)} numerical features:")
print(numerical_cols)
print(f"\nIdentified {len(categorical_cols)} categorical features:")
print(categorical_cols)

# 5. Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData split into training and testing sets:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

Feature 'pet_policy_binary' created.

Identified 9 numerical features:
['bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary']

Identified 2 categorical features:
['property_name', 'district']

Data split into training and testing sets:
X_train shape: (4924, 11)
X_test shape: (1232, 11)


In [10]:
# --- Model Evaluation Function ---

# This helper function will calculate all our performance metrics
# and return them in a dictionary.
# We store 'y_pred' so we can use it for our final CSV.

def evaluate_model(model, X_test, y_test, model_name):
    """Calculates R2, RMSE, and Accuracy metrics for a trained model."""
    
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = math.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calculate percentage error for accuracy
    percent_error = np.abs((y_test - y_pred) / y_test)
    accuracy_10 = np.mean(percent_error < 0.1) * 100  # Within 10%
    accuracy_20 = np.mean(percent_error < 0.2) * 100  # Within 20%
    accuracy_30 = np.mean(percent_error < 0.3) * 100  # Within 30%
    
    print(f"\n--- {model_name} Performance ---")
    print(f"R-squared: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"Accuracy (error < 10%): {accuracy_10:.2f}%")
    print(f"Accuracy (error < 20%): {accuracy_20:.2f}%")
    print(f"Accuracy (error < 30%): {accuracy_30:.2f}%")
    
    # Return results as a dictionary
    return {
        'model_name': model_name,
        'r2': r2,
        'rmse': rmse,
        'accuracy_10': accuracy_10,
        'accuracy_20': accuracy_20,
        'accuracy_30': accuracy_30,
        'y_pred': y_pred
    }

In [11]:
# --- Model Evaluation Function ---

# This helper function will calculate all our performance metrics
# and return them in a dictionary.
# We store 'y_pred' so we can use it for our final CSV.

def evaluate_model(model, X_test, y_test, model_name):
    """Calculates R2, RMSE, and Accuracy metrics for a trained model."""
    
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = math.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calculate percentage error for accuracy
    percent_error = np.abs((y_test - y_pred) / y_test)
    accuracy_10 = np.mean(percent_error < 0.1) * 100  # Within 10%
    accuracy_20 = np.mean(percent_error < 0.2) * 100  # Within 20%
    accuracy_30 = np.mean(percent_error < 0.3) * 100  # Within 30%
    
    print(f"\n--- {model_name} Performance ---")
    print(f"R-squared: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"Accuracy (error < 10%): {accuracy_10:.2f}%")
    print(f"Accuracy (error < 20%): {accuracy_20:.2f}%")
    print(f"Accuracy (error < 30%): {accuracy_30:.2f}%")
    
    # Return results as a dictionary
    return {
        'model_name': model_name,
        'r2': r2,
        'rmse': rmse,
        'accuracy_10': accuracy_10,
        'accuracy_20': accuracy_20,
        'accuracy_30': accuracy_30,
        'y_pred': y_pred  # Store predictions for final CSV
    }

In [12]:
# --- General Preprocessor (for RF, SVR, XGBoost) ---

# This preprocessor will be used by models that require
# explicit One-Hot Encoding for categorical data.

# Pipeline for numerical features:
# 1. Impute missing values with the median
# 2. Scale features using StandardScaler (very important for SVR)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical features:
# 1. Impute missing values with the string 'missing'
# 2. One-Hot Encode the features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine these transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    verbose_feature_names_out=False
)

print("General preprocessor (with One-Hot Encoding) created.")

#keknya gabutuh

General preprocessor (with One-Hot Encoding) created.


In [23]:
#new


In [14]:
from sklearn.model_selection import train_test_split
import numpy as np

# STEP 1: Clean your features first
print("=== Feature Cleaning ===")
print("Current X columns:", list(X_categorical.columns))

if 'property_name' in X_categorical.columns:
    X_categorical = X_categorical.drop('property_name', axis=1)
    print("✅ Removed property_name from features")

if 'distance_to_nearest_mtr_km' in X_categorical.columns:
    X_categorical = X_categorical.drop('distance_to_nearest_mtr_km', axis=1)
    print("✅ Removed distance_to_nearest_mtr_km from features")

# Remove additional columns
columns_to_drop = [
    'latitude', 
    'longitude', 
    'walking_time_to_mtr',
    'category_Community_Facilities_within_1000m', 
    'distance_to_nearest_match_km', 
    'distance_to_nearest_mtr_km', 
    'category_Recreation_within_1000m', 
    'category_Religion_within_2000m', 
    'category_Tourism_within_2000m', 
    'category_Transportation_within_1000m'
]

# Drop columns that exist in the dataframe
columns_dropped = []
for col in columns_to_drop:
    if col in X_categorical.columns:
        X_categorical = X_categorical.drop(col, axis=1)
        columns_dropped.append(col)

if columns_dropped:
    print(f"✅ Removed {len(columns_dropped)} additional columns: {', '.join(columns_dropped)}")
else:
    print("ℹ️ None of the additional columns were found in the dataframe")

if 'pet_policy' in X_categorical.columns:
    X_categorical['pet_policy_binary'] = X_categorical['pet_policy'].notna().astype(int)
    X_categorical = X_categorical.drop('pet_policy', axis=1)
    print("Feature 'pet_policy_binary' created.")

print("Final X columns:", list(X_categorical.columns))
print(f"X shape: {X_categorical.shape}")

# STEP 2: Three-way data split
print("\n=== Creating Three-Way Data Split ===")

# First split: separate out test set (20% of total data)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_categorical, 
    y_categorical, 
    test_size=0.2,      # 20% for final testing
    random_state=42
)

# Second split: divide remaining 80% into train (60% of total) and validation (20% of total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, 
    y_temp, 
    test_size=0.25,     # 25% of 80% = 20% of total for validation
    random_state=42
)

# Display the split sizes
total_samples = len(X_categorical)
print(f"\nTotal samples: {total_samples:,}")
print(f"Training set: {len(X_train):,} samples ({len(X_train)/total_samples*100:.1f}%)")
print(f"Validation set: {len(X_val):,} samples ({len(X_val)/total_samples*100:.1f}%)")
print(f"Test set: {len(X_test):,} samples ({len(X_test)/total_samples*100:.1f}%)")

# Verify the split adds up correctly
assert len(X_train) + len(X_val) + len(X_test) == total_samples
print(f"✅ Split verification: {len(X_train)} + {len(X_val)} + {len(X_test)} = {total_samples}")

# Check target variable distribution across splits
print(f"\n=== Target Variable Distribution ===")
print(f"Training set - Mean: ${y_train.mean():,.2f}, Std: ${y_train.std():,.2f}")
print(f"Validation set - Mean: ${y_val.mean():,.2f}, Std: ${y_val.std():,.2f}")
print(f"Test set - Mean: ${y_test.mean():,.2f}, Std: ${y_test.std():,.2f}")
print(f"Original data - Mean: ${y_categorical.mean():,.2f}, Std: ${y_categorical.std():,.2f}")

# Check feature consistency across splits
print(f"\n=== Feature Consistency Check ===")
print(f"All splits have same features: {list(X_train.columns) == list(X_val.columns) == list(X_test.columns)}")
print(f"Number of features: {X_train.shape[1]}")

print(f"\n🎯 Data is now properly split and ready for modeling!")
print(f"Next steps:")
print(f"1. Use X_train/y_train for training")
print(f"2. Use X_val/y_val for hyperparameter optimization")
print(f"3. Use X_test/y_test ONLY for final evaluation")

=== Feature Cleaning ===
Current X columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary']
ℹ️ None of the additional columns were found in the dataframe
Final X columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary']
X shape: (6156, 10)

=== Creating Three-Way Data Split ===

Total samples: 6,156
Training set: 3,693 samples (60.0%)
Validation set: 1,231 samples (20.0%)
Test set: 1,232 samples (20.0%)
✅ Split verification: 3693 + 1231 + 1232 = 6156

=== Target Variable Distribution ===
Training set - Mean: $8,899,962.09, Std: $5,112,105.30
Validation set - Mean: $8,811,689.68, Std: $4,790,063.20
Test set - Mea

In [48]:
X_train.columns

Index(['district', 'bedroom_count', 'property_age', 'saleable_area',
       'travel_time_to_cbd', 'total_poi_within_1000m',
       'category_Education_within_2000m', 'category_Medical_within_2000m',
       'category_Public_Market_within_1000m', 'pet_policy_binary'],
      dtype='object')

In [15]:
import optuna
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def lgb_objective(trial):
    # Test L1 vs L2 regularization with 2.5-5.0 range
    reg_alpha = trial.suggest_float('reg_alpha', 2.5, 5.0)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 2.5, 5.0)  # L2 regularization
    
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 150),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 15, 35),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
        'max_depth': trial.suggest_int('max_depth', 5, 7),
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'objective': 'regression',
        'random_state': 42,
        'verbosity': -1,
        'categorical_feature': 'auto'
    }
    
    try:
        model = lgb.LGBMRegressor(**params)
        
        # ✅ CHANGE 1: Train on training set only
        model.fit(X_train, y_train)
        
        # ✅ CHANGE 2: Evaluate on validation set (not training!)
        val_predictions = model.predict(X_val)
        val_mse = mean_squared_error(y_val, val_predictions)
        
        # ✅ CHANGE 3: Return negative MSE for maximization
        return -val_mse
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('-inf')

# Progress callback
def print_progress(study, trial):
    if trial.number % 25 == 0:
        # ✅ CHANGE 4: Show validation RMSE
        val_rmse = np.sqrt(-study.best_value)
        print(f"Completed {trial.number + 1} trials. Best validation RMSE: ${val_rmse:,.2f}")

print("Starting LightGBM hyperparameter optimization with L1/L2 regularization...")
print("🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)")  # ✅ CHANGE 5
print("Testing 100 different parameter combinations...")
print("Parameter ranges:")
print("  num_leaves: 31-150")
print("  min_data_in_leaf: 15-35") 
print("  feature_fraction: 0.7-1.0")
print("  n_estimators: 100-300")
print("  learning_rate: 0.05-0.2")
print("  max_depth: 5-7")
print("  reg_alpha (L1): 2.5-5.0")
print("  reg_lambda (L2): 2.5-5.0")

print(f"\nUsing proper train/validation/test splits:")
print(f"  Training samples: {len(X_train):,}")
print(f"  Validation samples: {len(X_val):,}")
print(f"  Test samples: {len(X_test):,}")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.NopPruner()
)

study.optimize(
    lgb_objective, 
    n_trials=100,
    timeout=7200,
    callbacks=[print_progress],
    show_progress_bar=True,
    n_jobs=1
)

# ✅ CHANGE 6: Updated results display
best_val_rmse = np.sqrt(-study.best_value)
print(f"\n=== LightGBM Optuna Results (Optimized for Validation) ===")
print(f"Completed {len(study.trials)} out of 100 trials")
print("Best trial:")
print(f"  Trial number: {study.best_trial.number}")
print(f"  Value (neg_MSE): {study.best_trial.value}")
print(f"  Validation RMSE: ${best_val_rmse:,.2f}")
print("  Best params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# Regularization analysis
best_l1 = study.best_trial.params['reg_alpha']
best_l2 = study.best_trial.params['reg_lambda']
print(f"\n=== Regularization Analysis ===")
print(f"Best L1 (reg_alpha): {best_l1:.4f}")
print(f"Best L2 (reg_lambda): {best_l2:.4f}")
print(f"Total regularization: {best_l1 + best_l2:.4f}")

if best_l1 > best_l2 * 1.5:
    print("✨ L1 regularization strongly preferred (feature selection focus)")
elif best_l2 > best_l1 * 1.5:
    print("✨ L2 regularization strongly preferred (weight shrinkage focus)")
elif best_l1 > 0.1 and best_l2 > 0.1:
    print("✨ Mixed L1+L2 regularization preferred (Elastic Net)")
else:
    print("✨ Minimal regularization preferred")

print("\nTraining final LightGBM model with best parameters...")
best_lgb_model = lgb.LGBMRegressor(
    **study.best_trial.params, 
    objective='regression',
    random_state=42,
    verbosity=-1,
    categorical_feature='auto'
)

# Fit on training data only
best_lgb_model.fit(X_train, y_train)

# Get predictions for all three sets
train_predictions = best_lgb_model.predict(X_train)
val_predictions = best_lgb_model.predict(X_val)
test_predictions = best_lgb_model.predict(X_test)

# Calculate metrics for all three sets
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

train_mae = mean_absolute_error(y_train, train_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)

train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Display performance across all sets
print(f"\n=== Model Performance Across All Sets ===")
print(f"Training Set:")
print(f"  RMSE: ${train_rmse:,.2f}")
print(f"  MAE: ${train_mae:,.2f}")
print(f"  R²: {train_r2:.4f}")

print(f"\nValidation Set:")
print(f"  RMSE: ${val_rmse:,.2f}")
print(f"  MAE: ${val_mae:,.2f}")
print(f"  R²: {val_r2:.4f}")

print(f"\nTest Set (UNSEEN DATA):")
print(f"  RMSE: ${test_rmse:,.2f}")
print(f"  MAE: ${test_mae:,.2f}")
print(f"  R²: {test_r2:.4f}")

# Check for overfitting
print(f"\n=== Overfitting Analysis ===")
if train_rmse < val_rmse * 0.8:
    print(f"⚠️  WARNING: Possible overfitting detected!")
    print(f"   Training RMSE ({train_rmse:,.2f}) much lower than validation RMSE ({val_rmse:,.2f})")
else:
    print(f"✅ Model generalizes well - training and validation performance similar")

if abs(val_rmse - test_rmse) / val_rmse < 0.1:
    print(f"✅ Validation set is good proxy for test performance")
else:
    print(f"⚠️  Large difference between validation and test performance")

# Create results with TEST set only (true unseen data)
results_df = X_test.copy()
results_df['actual_price'] = y_test
results_df['lgb_predicted_price'] = test_predictions
results_df['prediction_error'] = results_df['actual_price'] - results_df['lgb_predicted_price']
results_df['absolute_error'] = abs(results_df['prediction_error'])

# Save TEST ONLY results
results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/lgb_predictions_regularized.csv', index=False)

# ✅ NEW: Create combined results with ALL data (train + val + test)
train_df = X_train.copy()
train_df['dataset'] = 'train'
train_df['actual_price'] = y_train
train_df['lgb_predicted_price'] = train_predictions
train_df['prediction_error'] = train_df['actual_price'] - train_df['lgb_predicted_price']
train_df['absolute_error'] = abs(train_df['prediction_error'])

val_df = X_val.copy()
val_df['dataset'] = 'validation'
val_df['actual_price'] = y_val
val_df['lgb_predicted_price'] = val_predictions
val_df['prediction_error'] = val_df['actual_price'] - val_df['lgb_predicted_price']
val_df['absolute_error'] = abs(val_df['prediction_error'])

test_df = X_test.copy()
test_df['dataset'] = 'test'
test_df['actual_price'] = y_test
test_df['lgb_predicted_price'] = test_predictions
test_df['prediction_error'] = test_df['actual_price'] - test_df['lgb_predicted_price']
test_df['absolute_error'] = abs(test_df['prediction_error'])

# Combine all three datasets
all_results_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Save COMBINED results
all_results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/lgb_predictions_all_data.csv', index=False)

print(f"\n✅ Saved two CSV files:")
print(f"   1. Test only: lgb_predictions_regularized.csv ({len(results_df):,} rows)")
print(f"   2. All data: lgb_predictions_all_data.csv ({len(all_results_df):,} rows)")

print(f"\n=== Feature Importance Analysis ===")
feature_importance = best_lgb_model.feature_importances_
feature_names = X_train.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Top features by importance:")
for idx, row in importance_df.head(10).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100
importance_df.to_csv('/Users/clarencemarvin/Downloads/regularized2/lgb_feature_importance.csv', index=False)

# Trial summary
failed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]
print(f"\nTrial Summary:")
print(f"  Total trials: {len(study.trials)}")
print(f"  Successful trials: {len(study.trials) - len(failed_trials)}")
print(f"  Failed trials: {len(failed_trials)}")

print(f"\n🎯 FINAL RESULT: Test RMSE ${test_rmse:,.2f} on truly unseen data!")

[I 2025-11-24 20:53:18,006] A new study created in memory with name: no-name-2fc46dfc-9629-4492-9aa3-1146e2ef2d1a


Starting LightGBM hyperparameter optimization with L1/L2 regularization...
🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)
Testing 100 different parameter combinations...
Parameter ranges:
  num_leaves: 31-150
  min_data_in_leaf: 15-35
  feature_fraction: 0.7-1.0
  n_estimators: 100-300
  learning_rate: 0.05-0.2
  max_depth: 5-7
  reg_alpha (L1): 2.5-5.0
  reg_lambda (L2): 2.5-5.0

Using proper train/validation/test splits:
  Training samples: 3,693
  Validation samples: 1,231
  Test samples: 1,232


Best trial: 0. Best value: -2.71105e+12:   1%|          | 1/100 [00:00<01:20,  1.24it/s, 0.81/7200 seconds]

[I 2025-11-24 20:53:18,822] Trial 0 finished with value: -2711048151192.732 and parameters: {'reg_alpha': 2.61605014760634, 'reg_lambda': 3.434584137704446, 'max_depth': 7, 'num_leaves': 72, 'min_data_in_leaf': 28, 'feature_fraction': 0.8706648511635897, 'n_estimators': 257, 'learning_rate': 0.11846793264367296}. Best is trial 0 with value: -2711048151192.732.
Completed 1 trials. Best validation RMSE: $1,646,526.09


Best trial: 1. Best value: -2.63204e+12:   2%|▏         | 2/100 [00:01<00:48,  2.03it/s, 1.08/7200 seconds]

...

[I 2025-11-24 20:54:30,617] Trial 97 finished with value: -2675276820673.502 and parameters: {'reg_alpha': 2.6525789535529496, 'reg_lambda': 3.2657980642489335, 'max_depth': 6, 'num_leaves': 46, 'min_data_in_leaf': 30, 'feature_fraction': 0.8180317862009517, 'n_estimators': 277, 'learning_rate': 0.09085506048773409}. Best is trial 76 with value: -2405087905258.0024.


Best trial: 76. Best value: -2.40509e+12:  99%|█████████▉| 99/100 [01:13<00:01,  1.02s/it, 73.64/7200 seconds]

[I 2025-11-24 20:54:31,656] Trial 98 finished with value: -2549024201818.788 and parameters: {'reg_alpha': 3.0012904864641086, 'reg_lambda': 3.45900176081594, 'max_depth': 6, 'num_leaves': 41, 'min_data_in_leaf': 18, 'feature_fraction': 0.8427121859571446, 'n_estimators': 292, 'learning_rate': 0.08756019314343683}. Best is trial 76 with value: -2405087905258.0024.


Best trial: 76. Best value: -2.40509e+12: 100%|██████████| 100/100 [01:15<00:00,  1.32it/s, 75.49/7200 seconds]


[I 2025-11-24 20:54:33,505] Trial 99 finished with value: -2484754499796.9434 and parameters: {'reg_alpha': 2.7760411941427856, 'reg_lambda': 2.964813980095601, 'max_depth': 6, 'num_leaves': 45, 'min_data_in_leaf': 17, 'feature_fraction': 0.8231050919293389, 'n_estimators': 268, 'learning_rate': 0.06657417310238027}. Best is trial 76 with value: -2405087905258.0024.

=== LightGBM Optuna Results (Optimized for Validation) ===
Completed 100 out of 100 trials
Best trial:
  Trial number: 76
  Value (neg_MSE): -2405087905258.0024
  Validation RMSE: $1,550,834.58
  Best params:
    reg_alpha: 3.096882141506273
    reg_lambda: 4.0226004436973675
    max_depth: 6
    num_leaves: 49
    min_data_in_leaf: 18
    feature_fraction: 0.8901570382889373
    n_estimators: 262
    learning_rate: 0.07417854704442373

=== Regularization Analysis ===
Best L1 (reg_alpha): 3.0969
Best L2 (reg_lambda): 4.0226
Total regularization: 7.1195
✨ Mixed L1+L2 regularization preferred (Elastic Net)

Training final Li

In [50]:
import optuna
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def xgb_objective(trial):
    reg_alpha = trial.suggest_float('reg_alpha', 2.5, 5.0)
    reg_lambda = trial.suggest_float('reg_lambda', 2.5, 5.0)
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.85),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'objective': 'reg:squarederror',
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'enable_categorical': True
    }
    
    try:
        model = xgb.XGBRegressor(**params)
        
        # ✅ CHANGE 1: Train on training set only
        model.fit(X_train, y_train)
        
        # ✅ CHANGE 2: Evaluate on validation set (not training!)
        val_predictions = model.predict(X_val)
        val_mse = mean_squared_error(y_val, val_predictions)
        
        # ✅ CHANGE 3: Return negative MSE for maximization
        return -val_mse
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('-inf')

def print_progress(study, trial):
    if trial.number % 25 == 0:
        # ✅ CHANGE 4: Show validation RMSE
        val_rmse = np.sqrt(-study.best_value)
        print(f"Completed {trial.number + 1} trials. Best validation RMSE: ${val_rmse:,.2f}")

print("Starting XGBoost hyperparameter optimization...")
print("🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)")  # ✅ CHANGE 5
print(f"Training samples: {len(X_train):,}")

study = optuna.create_study(direction='maximize')
study.optimize(
    xgb_objective, 
    n_trials=100,
    callbacks=[print_progress],
    show_progress_bar=True
)

# ✅ CHANGE 6: Updated results display
best_val_rmse = np.sqrt(-study.best_value)
print(f"\n=== XGBoost Results (Optimized for Validation) ===")
print(f"Best validation RMSE: ${best_val_rmse:,.2f}")
print("Best params:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

# Regularization analysis
best_l1 = study.best_trial.params['reg_alpha']
best_l2 = study.best_trial.params['reg_lambda']
print(f"\nRegularization: L1={best_l1:.3f}, L2={best_l2:.3f}")

# Train final model
best_xgb_model = xgb.XGBRegressor(
    **study.best_trial.params, 
    objective='reg:squarederror',
    random_state=42,
    verbosity=0,
    n_jobs=-1,
    enable_categorical=True
)

best_xgb_model.fit(X_train, y_train)

# Get predictions for all sets
train_predictions = best_xgb_model.predict(X_train)
val_predictions = best_xgb_model.predict(X_val)
test_predictions = best_xgb_model.predict(X_test)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"\n=== Performance ===")
print(f"Training:   RMSE ${train_rmse:,.2f}, R² {train_r2:.4f}")
print(f"Validation: RMSE ${val_rmse:,.2f}, R² {val_r2:.4f}")
print(f"Test:       RMSE ${test_rmse:,.2f}, R² {test_r2:.4f}")

# Overfitting check
if train_rmse < val_rmse * 0.8:
    print(f"⚠️  Possible overfitting detected")
else:
    print(f"✅ Good generalization")

# Create results with TEST set only (true unseen data)
results_df = X_test.copy()
results_df['actual_price'] = y_test
results_df['xgb_predicted_price'] = test_predictions
results_df['prediction_error'] = results_df['actual_price'] - results_df['xgb_predicted_price']
results_df['absolute_error'] = abs(results_df['prediction_error'])

# Save TEST ONLY results
results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/xgb_predictions_regularized.csv', index=False)

# ✅ NEW: Create combined results with ALL data (train + val + test)
train_df = X_train.copy()
train_df['dataset'] = 'train'
train_df['actual_price'] = y_train
train_df['xgb_predicted_price'] = train_predictions
train_df['prediction_error'] = train_df['actual_price'] - train_df['xgb_predicted_price']
train_df['absolute_error'] = abs(train_df['prediction_error'])

val_df = X_val.copy()
val_df['dataset'] = 'validation'
val_df['actual_price'] = y_val
val_df['xgb_predicted_price'] = val_predictions
val_df['prediction_error'] = val_df['actual_price'] - val_df['xgb_predicted_price']
val_df['absolute_error'] = abs(val_df['prediction_error'])

test_df = X_test.copy()
test_df['dataset'] = 'test'
test_df['actual_price'] = y_test
test_df['xgb_predicted_price'] = test_predictions
test_df['prediction_error'] = test_df['actual_price'] - test_df['xgb_predicted_price']
test_df['absolute_error'] = abs(test_df['prediction_error'])

# Combine all three datasets
all_results_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Save COMBINED results
all_results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/xgb_predictions_all_data.csv', index=False)

print(f"\n✅ Saved two CSV files:")
print(f"   1. Test only: lgb_predictions_regularized.csv ({len(results_df):,} rows)")
print(f"   2. All data: lgb_predictions_all_data.csv ({len(all_results_df):,} rows)")

# Feature importance
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100

print(f"\n=== Top Features ===")
for idx, row in importance_df.head(10).iterrows():
    print(f"{row['feature']}: {row['importance_pct']:.1f}%")

importance_df.to_csv('/Users/clarencemarvin/Downloads/regularized2/xgb_feature_importance.csv', index=False)

print(f"\nFinal test RMSE: ${test_rmse:,.2f}")

[I 2025-11-11 12:50:26,499] A new study created in memory with name: no-name-a60f551c-f205-4e1e-9c5c-fcf8d697c68a


Starting XGBoost hyperparameter optimization...
🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)
Training samples: 3,693


Best trial: 0. Best value: -2.6686e+12:   1%|          | 1/100 [00:00<01:32,  1.07it/s]

[I 2025-11-11 12:50:27,439] Trial 0 finished with value: -2668601802752.0 and parameters: {'reg_alpha': 3.5179657189844993, 'reg_lambda': 4.772757441512988, 'max_depth': 7, 'min_child_weight': 7, 'subsample': 0.7861611456586872, 'colsample_bytree': 0.8458964844682395, 'learning_rate': 0.2909076896438328, 'n_estimators': 109}. Best is trial 0 with value: -2668601802752.0.
Completed 1 trials. Best validation RMSE: $1,633,585.57


Best trial: 1. Best value: -2.48223e+12:   2%|▏         | 2/100 [00:01<01:26,  1.13it/s]

...

[I 2025-11-11 12:51:11,943] Trial 97 finished with value: -2372233330688.0 and parameters: {'reg_alpha': 3.6547688049931706, 'reg_lambda': 3.1895420970756097, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.9601629001033741, 'colsample_bytree': 0.7337447318371207, 'learning_rate': 0.0655467355605208, 'n_estimators': 180}. Best is trial 95 with value: -2329292046336.0.


Best trial: 95. Best value: -2.32929e+12:  99%|█████████▉| 99/100 [00:45<00:00,  2.40it/s]

[I 2025-11-11 12:51:12,377] Trial 98 finished with value: -2559537053696.0 and parameters: {'reg_alpha': 3.660383763652677, 'reg_lambda': 3.009652715836197, 'max_depth': 4, 'min_child_weight': 3, 'subsample': 0.9760727812715254, 'colsample_bytree': 0.7210103626333216, 'learning_rate': 0.06497604597928144, 'n_estimators': 183}. Best is trial 95 with value: -2329292046336.0.


Best trial: 99. Best value: -2.31135e+12: 100%|██████████| 100/100 [00:46<00:00,  2.16it/s]


[I 2025-11-11 12:51:12,865] Trial 99 finished with value: -2311348813824.0 and parameters: {'reg_alpha': 3.826317569224855, 'reg_lambda': 3.168769064516178, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.9535895592211322, 'colsample_bytree': 0.7176601248730557, 'learning_rate': 0.046684073520362704, 'n_estimators': 189}. Best is trial 99 with value: -2311348813824.0.

=== XGBoost Results (Optimized for Validation) ===
Best validation RMSE: $1,520,312.08
Best params:
  reg_alpha: 3.826317569224855
  reg_lambda: 3.168769064516178
  max_depth: 5
  min_child_weight: 2
  subsample: 0.9535895592211322
  colsample_bytree: 0.7176601248730557
  learning_rate: 0.046684073520362704
  n_estimators: 189

Regularization: L1=3.826, L2=3.169

=== Performance ===
Training:   RMSE $1,107,670.38, R² 0.9530
Validation: RMSE $1,520,312.08, R² 0.8992
Test:       RMSE $2,006,796.92, R² 0.8456
⚠️  Possible overfitting detected

✅ Saved two CSV files:
   1. Test only: lgb_predictions_regularized.csv (1,

In [51]:
import optuna
from sklearn.model_selection import cross_val_score
import catboost as cb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

def catboost_objective(trial):
    # CatBoost uses different regularization parameter names
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1.0, 10.0)  # L2 regularization
    
    params = {
        'depth': trial.suggest_int('depth', 4, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.15),
        'iterations': trial.suggest_int('iterations', 100, 300),
        'l2_leaf_reg': l2_leaf_reg,  # Use CatBoost's L2 parameter name
        'border_count': trial.suggest_int('border_count', 32, 128),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 2.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        
        # Fixed parameters
        'objective': 'RMSE',
        'random_state': 42,
        'verbose': False,
        'thread_count': -1,
        'cat_features': ['district']  # Specify categorical features
    }
    
    try:
        model = cb.CatBoostRegressor(**params)
        
        # ✅ CHANGE 1: Train on training set only
        model.fit(X_train, y_train)
        
        # ✅ CHANGE 2: Evaluate on validation set (not training!)
        val_predictions = model.predict(X_val)
        val_mse = mean_squared_error(y_val, val_predictions)
        
        # ✅ CHANGE 3: Return negative MSE for maximization (Optuna maximizes)
        return -val_mse
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('-inf')

def print_progress(study, trial):
    if trial.number % 25 == 0:
        # ✅ CHANGE 4: Show validation RMSE instead of raw score
        val_rmse = np.sqrt(-study.best_value)
        print(f"Completed {trial.number + 1} trials. Best validation RMSE: ${val_rmse:,.2f}")

print("Starting CatBoost hyperparameter optimization...")
print("🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)")  # ✅ CHANGE 5: Clear messaging
print(f"Training samples: {len(X_train):,}")

study = optuna.create_study(direction='maximize')
study.optimize(
    catboost_objective, 
    n_trials=100,
    callbacks=[print_progress],
    show_progress_bar=True
)

# ✅ CHANGE 6: Updated results display
best_val_rmse = np.sqrt(-study.best_value)
print(f"\n=== CatBoost Results (Optimized for Validation) ===")
print(f"Best validation RMSE: ${best_val_rmse:,.2f}")
print("Best params:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

# Train final model with corrected parameters
best_params = study.best_trial.params.copy()
best_params.update({
    'objective': 'RMSE',
    'random_state': 42,
    'verbose': False,
    'thread_count': -1,
    'cat_features': ['district']
})

best_catboost_model = cb.CatBoostRegressor(**best_params)

# Fit on training data only
best_catboost_model.fit(X_train, y_train)

# Get predictions for all sets
train_predictions = best_catboost_model.predict(X_train)
val_predictions = best_catboost_model.predict(X_val)
test_predictions = best_catboost_model.predict(X_test)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"\n=== Performance ===")
print(f"Training:   RMSE ${train_rmse:,.2f}, R² {train_r2:.4f}")
print(f"Validation: RMSE ${val_rmse:,.2f}, R² {val_r2:.4f}")
print(f"Test:       RMSE ${test_rmse:,.2f}, R² {test_r2:.4f}")

# Overfitting check
if train_rmse < val_rmse * 0.8:
    print(f"⚠️  Possible overfitting detected")
else:
    print(f"✅ Good generalization")


# Create results with TEST set only (true unseen data)
results_df = X_test.copy()
results_df['actual_price'] = y_test
results_df['catboost_predicted_price'] = test_predictions
results_df['prediction_error'] = results_df['actual_price'] - results_df['catboost_predicted_price']
results_df['absolute_error'] = abs(results_df['prediction_error'])

# Save TEST ONLY results
results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/catboost_predicted_price.csv', index=False)

# ✅ NEW: Create combined results with ALL data (train + val + test)
train_df = X_train.copy()
train_df['dataset'] = 'train'
train_df['actual_price'] = y_train
train_df['catboost_predicted_price'] = train_predictions
train_df['prediction_error'] = train_df['actual_price'] - train_df['catboost_predicted_price']
train_df['absolute_error'] = abs(train_df['prediction_error'])

val_df = X_val.copy()
val_df['dataset'] = 'validation'
val_df['actual_price'] = y_val
val_df['catboost_predicted_price'] = val_predictions
val_df['prediction_error'] = val_df['actual_price'] - val_df['catboost_predicted_price']
val_df['absolute_error'] = abs(val_df['prediction_error'])

test_df = X_test.copy()
test_df['dataset'] = 'test'
test_df['actual_price'] = y_test
test_df['catboost_predicted_price'] = test_predictions
test_df['prediction_error'] = test_df['actual_price'] - test_df['catboost_predicted_price']
test_df['absolute_error'] = abs(test_df['prediction_error'])

# Combine all three datasets
all_results_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Save COMBINED results
all_results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/catboost_predictions_all_data.csv', index=False)

print(f"\n✅ Saved two CSV files:")
print(f"   1. Test only: lgb_predictions_regularized.csv ({len(results_df):,} rows)")
print(f"   2. All data: lgb_predictions_all_data.csv ({len(all_results_df):,} rows)")

# Feature importance
feature_names = [col for col in X_train.columns if col != 'district'] + ['district']
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': best_catboost_model.feature_importances_
}).sort_values('importance', ascending=False)

importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100

print(f"\n=== Top Features ===")
for idx, row in importance_df.head(10).iterrows():
    print(f"{row['feature']}: {row['importance_pct']:.1f}%")

importance_df.to_csv('/Users/clarencemarvin/Downloads/regularized2/catboost_feature_importance.csv', index=False)

print(f"\nFinal test RMSE: ${test_rmse:,.2f}")

[I 2025-11-11 12:51:44,727] A new study created in memory with name: no-name-9084f875-89c2-4315-bd50-d8f3a24114e7


Starting CatBoost hyperparameter optimization...
🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)
Training samples: 3,693


Best trial: 0. Best value: -2.82937e+12:   2%|▏         | 2/100 [00:01<00:53,  1.85it/s]

[I 2025-11-11 12:51:45,815] Trial 0 finished with value: -2829368688887.069 and parameters: {'l2_leaf_reg': 3.5435971033157525, 'depth': 8, 'learning_rate': 0.049105786732791666, 'iterations': 292, 'border_count': 38, 'bagging_temperature': 0.8354598338078925, 'random_strength': 1.1556698031184143, 'min_data_in_leaf': 33}. Best is trial 0 with value: -2829368688887.069.
Completed 1 trials. Best validation RMSE: $1,682,072.74
[I 2025-11-11 12:51:45,973] Trial 1 finished with value: -3059000157999.721 and parameters: {'l2_leaf_reg': 5.627436551428138, 'depth': 5, 'learning_rate': 0.12586608602349572, 'iterations': 135, 'border_count': 68, 'bagging_temperature': 0.7135468402451469, 'random_strength': 0.9912498582615117, 'min_data_in_leaf': 26}. Best is trial 0 with value: -2829368688887.069.


Best trial: 0. Best value: -2.82937e+12:   3%|▎         | 3/100 [00:01<00:37,  2.60it/s]

...

[I 2025-11-11 12:53:04,719] Trial 97 finished with value: -2686624872795.7256 and parameters: {'l2_leaf_reg': 2.258180619569552, 'depth': 8, 'learning_rate': 0.12213608913128615, 'iterations': 278, 'border_count': 95, 'bagging_temperature': 0.705304459188597, 'random_strength': 0.8572400751048852, 'min_data_in_leaf': 78}. Best is trial 62 with value: -2536578785454.5786.


Best trial: 62. Best value: -2.53658e+12:  99%|█████████▉| 99/100 [01:20<00:00,  1.18it/s]

[I 2025-11-11 12:53:05,587] Trial 98 finished with value: -2773940346223.319 and parameters: {'l2_leaf_reg': 2.8414957307571, 'depth': 7, 'learning_rate': 0.14724933783880917, 'iterations': 286, 'border_count': 113, 'bagging_temperature': 0.3970932475347412, 'random_strength': 0.7845474685342476, 'min_data_in_leaf': 50}. Best is trial 62 with value: -2536578785454.5786.


Best trial: 62. Best value: -2.53658e+12: 100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


[I 2025-11-11 12:53:06,631] Trial 99 finished with value: -2598565541227.576 and parameters: {'l2_leaf_reg': 1.973947793115125, 'depth': 8, 'learning_rate': 0.1390253101435614, 'iterations': 300, 'border_count': 104, 'bagging_temperature': 0.3110531766854997, 'random_strength': 1.1791277764221055, 'min_data_in_leaf': 57}. Best is trial 62 with value: -2536578785454.5786.

=== CatBoost Results (Optimized for Validation) ===
Best validation RMSE: $1,592,664.05
Best params:
  l2_leaf_reg: 1.3959841858247966
  depth: 7
  learning_rate: 0.12582254186448188
  iterations: 274
  border_count: 125
  bagging_temperature: 0.8410542555619
  random_strength: 0.6480231368678006
  min_data_in_leaf: 48

=== Performance ===
Training:   RMSE $1,086,688.53, R² 0.9548
Validation: RMSE $1,592,664.05, R² 0.8894
Test:       RMSE $2,032,497.50, R² 0.8417
⚠️  Possible overfitting detected

✅ Saved two CSV files:
   1. Test only: lgb_predictions_regularized.csv (1,232 rows)
   2. All data: lgb_predictions_all_d

In [52]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Prepare data for Random Forest with proper train/val/test splits
print(f"=== Random Forest Data Preparation ===")
print(f"Original data shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")
print(f"  X_test: {X_test.shape}")

# Encode district for all splits
X_train_rf = X_train.copy()
X_val_rf = X_val.copy()
X_test_rf = X_test.copy()

# Convert district to numeric codes for all splits
X_train_rf['district_encoded'] = X_train_rf['district'].cat.codes
X_val_rf['district_encoded'] = X_val_rf['district'].cat.codes
X_test_rf['district_encoded'] = X_test_rf['district'].cat.codes

# Remove original categorical district
X_train_rf = X_train_rf.drop('district', axis=1)
X_val_rf = X_val_rf.drop('district', axis=1)
X_test_rf = X_test_rf.drop('district', axis=1)

print(f"RF data shapes after encoding:")
print(f"  X_train_rf: {X_train_rf.shape}")
print(f"  X_val_rf: {X_val_rf.shape}")
print(f"  X_test_rf: {X_test_rf.shape}")
print(f"Features: {list(X_train_rf.columns)}")
print(f"District codes range: {X_train_rf['district_encoded'].min()} to {X_train_rf['district_encoded'].max()}")

def rf_objective(trial):
    # Test regularization via controlling complexity
    max_depth_choice = trial.suggest_categorical('max_depth_type', ['int', 'none'])
    if max_depth_choice == 'none':
        max_depth = None
    else:
        max_depth = trial.suggest_int('max_depth', 3, 12)
    
    # Regularization through tree complexity control
    min_samples_split = trial.suggest_int('min_samples_split', 10, 50)   # Increased for more regularization
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 20)     # Increased for more regularization
    max_leaf_nodes = trial.suggest_categorical('max_leaf_nodes_type', ['int', 'none'])
    if max_leaf_nodes == 'none':
        max_leaf_nodes_val = None
    else:
        max_leaf_nodes_val = trial.suggest_int('max_leaf_nodes', 10, 100)
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),     # More conservative range
        'max_features': trial.suggest_categorical('max_features', [0.3, 0.5, 0.7]),
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_leaf_nodes': max_leaf_nodes_val,
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        
        # Fixed parameters
        'random_state': 42,
        'n_jobs': -1
    }
    
    try:
        # Create Random Forest model
        model = RandomForestRegressor(**params)
        
        # Train on training set only
        model.fit(X_train_rf, y_train)
        
        # Evaluate on VALIDATION set (not training!)
        val_predictions = model.predict(X_val_rf)
        val_mse = mean_squared_error(y_val, val_predictions)
        
        # Return negative MSE for maximization (Optuna maximizes)
        return -val_mse
    
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('-inf')

def print_progress(study, trial):
    if trial.number % 25 == 0:
        val_rmse = np.sqrt(-study.best_value)
        print(f"Completed {trial.number + 1} trials. Best validation RMSE: ${val_rmse:,.2f}")

# Create Optuna study
print("\nStarting Random Forest hyperparameter optimization...")
print("🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)")
print("Using proper train/validation/test splits")
print("Testing 150 different parameter combinations...")
print("Regularization parameter ranges:")
print("  min_samples_split: 5-50 (higher = more regularization)")
print("  min_samples_leaf: 5-30 (higher = more regularization)")
print("  max_leaf_nodes: 10-100 or None (lower = more regularization)")

study = optuna.create_study(direction='maximize')
study.optimize(
    rf_objective, 
    n_trials=150,
    callbacks=[print_progress],
    show_progress_bar=True
)

# Print results
best_val_rmse = np.sqrt(-study.best_value)
print(f"\n=== Random Forest Optuna Results (Optimized for Validation) ===")
print(f"Completed {len(study.trials)} trials")
print("Best trial:")
print(f"  Value (neg_MSE): {study.best_value}")
print(f"  Validation RMSE: ${best_val_rmse:,.2f}")
print("  Best params:")
for key, value in study.best_trial.params.items():
    if not key.endswith('_type'):
        print(f"    {key}: {value}")

# Analyze regularization
best_min_split = study.best_trial.params['min_samples_split']
best_min_leaf = study.best_trial.params['min_samples_leaf']
best_max_leaf = study.best_trial.params.get('max_leaf_nodes', 'None')

print(f"\n=== Regularization Analysis ===")
print(f"min_samples_split: {best_min_split} (higher = more regularization)")
print(f"min_samples_leaf: {best_min_leaf} (higher = more regularization)")
print(f"max_leaf_nodes: {best_max_leaf} (lower = more regularization)")

reg_strength = (best_min_split - 5) + (best_min_leaf - 5)
if best_max_leaf != 'None' and best_max_leaf < 50:
    reg_strength += 2

if reg_strength > 20:
    print("✨ Strong regularization preferred (very conservative trees)")
elif reg_strength > 10:
    print("✨ Moderate regularization preferred (balanced complexity)")
else:
    print("✨ Light regularization preferred (more flexible trees)")

# Train final model with best parameters
print("\nTraining final Random Forest model with best parameters...")

best_params = study.best_trial.params.copy()
if best_params['max_depth_type'] == 'none':
    best_params['max_depth'] = None
if best_params.get('max_leaf_nodes_type') == 'none':
    best_params['max_leaf_nodes'] = None

# Remove helper parameters
best_params.pop('max_depth_type', None)
best_params.pop('max_leaf_nodes_type', None)

best_rf_model = RandomForestRegressor(**best_params, 
                                      random_state=42,
                                      n_jobs=-1)

# Fit on training data only
best_rf_model.fit(X_train_rf, y_train)

# Make predictions on all sets
train_predictions = best_rf_model.predict(X_train_rf)
val_predictions = best_rf_model.predict(X_val_rf)
test_predictions = best_rf_model.predict(X_test_rf)

# Calculate metrics for all sets
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

train_mae = mean_absolute_error(y_train, train_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)

train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"\n=== Model Performance Across All Sets ===")
print("Training Set:")
print(f"  RMSE: ${train_rmse:,.2f}")
print(f"  MAE: ${train_mae:,.2f}")
print(f"  R²: {train_r2:.4f}")

print("\nValidation Set:")
print(f"  RMSE: ${val_rmse:,.2f}")
print(f"  MAE: ${val_mae:,.2f}")
print(f"  R²: {val_r2:.4f}")

print("\nTest Set (UNSEEN DATA):")
print(f"  RMSE: ${test_rmse:,.2f}")
print(f"  MAE: ${test_mae:,.2f}")
print(f"  R²: {test_r2:.4f}")

# Overfitting analysis
print(f"\n=== Overfitting Analysis ===")
if train_rmse < val_rmse * 0.8:
    print(f"⚠️  WARNING: Possible overfitting detected!")
    print(f"   Training RMSE (${train_rmse:,.2f}) much lower than validation RMSE (${val_rmse:,.2f})")
elif train_rmse < val_rmse * 0.9:
    print(f"⚠️  Mild overfitting detected")
    print(f"   Training RMSE: ${train_rmse:,.2f}, Validation RMSE: ${val_rmse:,.2f}")
else:
    print(f"✅ Good generalization!")
    print(f"   Training RMSE: ${train_rmse:,.2f}, Validation RMSE: ${val_rmse:,.2f}")

if abs(val_rmse - test_rmse) > val_rmse * 0.2:
    print(f"⚠️  Large difference between validation and test performance")
else:
    print(f"✅ Consistent performance between validation and test sets")


# Create results with TEST set only (true unseen data)
results_df = X_test.copy()
results_df['actual_price'] = y_test
results_df['rf_predicted_price'] = test_predictions
results_df['prediction_error'] = results_df['actual_price'] - results_df['rf_predicted_price']
results_df['absolute_error'] = abs(results_df['prediction_error'])

# Save TEST ONLY results
results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/rf_predictions_regularized.csv', index=False)

# ✅ NEW: Create combined results with ALL data (train + val + test)
train_df = X_train.copy()
train_df['dataset'] = 'train'
train_df['actual_price'] = y_train
train_df['rf_predicted_price'] = train_predictions
train_df['prediction_error'] = train_df['actual_price'] - train_df['rf_predicted_price']
train_df['absolute_error'] = abs(train_df['prediction_error'])

val_df = X_val.copy()
val_df['dataset'] = 'validation'
val_df['actual_price'] = y_val
val_df['rf_predicted_price'] = val_predictions
val_df['prediction_error'] = val_df['actual_price'] - val_df['rf_predicted_price']
val_df['absolute_error'] = abs(val_df['prediction_error'])

test_df = X_test.copy()
test_df['dataset'] = 'test'
test_df['actual_price'] = y_test
test_df['rf_predicted_price'] = test_predictions
test_df['prediction_error'] = test_df['actual_price'] - test_df['rf_predicted_price']
test_df['absolute_error'] = abs(test_df['prediction_error'])

# Combine all three datasets
all_results_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Save COMBINED results
all_results_df.to_csv('/Users/clarencemarvin/Downloads/regularized/rf_predictions_regularized.csv', index=False)

print(f"\n✅ Saved two CSV files:")
print(f"   1. Test only: lgb_predictions_regularized.csv ({len(results_df):,} rows)")
print(f"   2. All data: lgb_predictions_all_data.csv ({len(all_results_df):,} rows)")

# Feature Importance Analysis
print(f"\n=== Feature Importance Analysis ===")
feature_importance = best_rf_model.feature_importances_
feature_names = X_train_rf.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100

print("Top 10 features by importance:")
for idx, row in importance_df.head(10).iterrows():
    print(f"  {row['feature']}: {row['importance_pct']:.2f}%")

importance_df.to_csv('/Users/clarencemarvin/Downloads/regularized2/rf_feature_importance.csv', index=False)

print(f"\nOptimization completed! Final test RMSE: ${test_rmse:,.2f}")
print(f"Results saved to CSV files.")

[I 2025-11-11 12:53:18,535] A new study created in memory with name: no-name-ec24c1b4-7af9-40f6-999b-013b331fb216


=== Random Forest Data Preparation ===
Original data shapes:
  X_train: (3693, 10)
  X_val: (1231, 10)
  X_test: (1232, 10)
RF data shapes after encoding:
  X_train_rf: (3693, 10)
  X_val_rf: (1231, 10)
  X_test_rf: (1232, 10)
Features: ['bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary', 'district_encoded']
District codes range: 0 to 143

Starting Random Forest hyperparameter optimization...
🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)
Using proper train/validation/test splits
Testing 150 different parameter combinations...
Regularization parameter ranges:
  min_samples_split: 5-50 (higher = more regularization)
  min_samples_leaf: 5-30 (higher = more regularization)
  max_leaf_nodes: 10-100 or None (lower = more regularization)


Best trial: 0. Best value: -4.18879e+12:   1%|          | 1/150 [00:00<00:32,  4.63it/s]

[I 2025-11-11 12:53:18,758] Trial 0 finished with value: -4188793726154.143 and parameters: {'max_depth_type': 'none', 'min_samples_split': 34, 'min_samples_leaf': 16, 'max_leaf_nodes_type': 'int', 'max_leaf_nodes': 92, 'n_estimators': 113, 'max_features': 0.3, 'bootstrap': False}. Best is trial 0 with value: -4188793726154.143.
Completed 1 trials. Best validation RMSE: $2,046,654.28


Best trial: 0. Best value: -4.18879e+12:   2%|▏         | 3/150 [00:00<00:33,  4.44it/s]

...

[I 2025-11-11 12:54:25,145] Trial 146 finished with value: -3875573834937.8613 and parameters: {'max_depth_type': 'none', 'min_samples_split': 13, 'min_samples_leaf': 20, 'max_leaf_nodes_type': 'none', 'n_estimators': 162, 'max_features': 0.5, 'bootstrap': False}. Best is trial 105 with value: -2866462622745.4785.


Best trial: 105. Best value: -2.86646e+12:  99%|█████████▊| 148/150 [01:06<00:00,  2.26it/s]

[I 2025-11-11 12:54:25,525] Trial 147 finished with value: -3624817908784.2114 and parameters: {'max_depth_type': 'none', 'min_samples_split': 12, 'min_samples_leaf': 6, 'max_leaf_nodes_type': 'int', 'max_leaf_nodes': 85, 'n_estimators': 151, 'max_features': 0.5, 'bootstrap': False}. Best is trial 105 with value: -2866462622745.4785.


Best trial: 105. Best value: -2.86646e+12: 100%|██████████| 150/150 [01:07<00:00,  2.22it/s]


[I 2025-11-11 12:54:25,987] Trial 148 finished with value: -2978329977205.8643 and parameters: {'max_depth_type': 'none', 'min_samples_split': 11, 'min_samples_leaf': 6, 'max_leaf_nodes_type': 'none', 'n_estimators': 155, 'max_features': 0.5, 'bootstrap': False}. Best is trial 105 with value: -2866462622745.4785.
[I 2025-11-11 12:54:26,172] Trial 149 finished with value: -3065896219427.446 and parameters: {'max_depth_type': 'none', 'min_samples_split': 14, 'min_samples_leaf': 5, 'max_leaf_nodes_type': 'none', 'n_estimators': 61, 'max_features': 0.5, 'bootstrap': False}. Best is trial 105 with value: -2866462622745.4785.

=== Random Forest Optuna Results (Optimized for Validation) ===
Completed 150 trials
Best trial:
  Value (neg_MSE): -2866462622745.4785
  Validation RMSE: $1,693,063.09
  Best params:
    min_samples_split: 10
    min_samples_leaf: 5
    n_estimators: 139
    max_features: 0.5
    bootstrap: False

=== Regularization Analysis ===
min_samples_split: 10 (higher = more re

In [53]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd

# ✅ CHANGE 1: Prepare data for SVR using proper splits
print(f"=== SVR Data Preparation ===")

# Prepare training data
X_svr_train = X_train.copy()
X_svr_train['district_encoded'] = X_svr_train['district'].cat.codes
X_svr_train = X_svr_train.drop('district', axis=1)

# Prepare validation data
X_svr_val = X_val.copy()
X_svr_val['district_encoded'] = X_svr_val['district'].cat.codes
X_svr_val = X_svr_val.drop('district', axis=1)

# Prepare test data
X_svr_test = X_test.copy()
X_svr_test['district_encoded'] = X_svr_test['district'].cat.codes
X_svr_test = X_svr_test.drop('district', axis=1)

print(f"Training data shape: {X_svr_train.shape}")
print(f"Validation data shape: {X_svr_val.shape}")
print(f"Test data shape: {X_svr_test.shape}")
print(f"Features: {list(X_svr_train.columns)}")

def svr_objective(trial):
    params = {
        'C': trial.suggest_float('C', 0.1, 1000, log=True),           # Regularization parameter
        'epsilon': trial.suggest_float('epsilon', 0.01, 1.0),         # Epsilon-tube width
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']), # Kernel coefficient
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'poly', 'linear']), # Kernel type
    }
    
    # Add degree parameter only for polynomial kernel
    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
    
    try:
        # ✅ CHANGE 2: Create SVR pipeline with scaling
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svr', SVR(**params))
        ])
        
        # ✅ CHANGE 3: Train on training set only
        pipeline.fit(X_svr_train, y_train)
        
        # ✅ CHANGE 4: Evaluate on validation set
        val_predictions = pipeline.predict(X_svr_val)
        val_mse = mean_squared_error(y_val, val_predictions)
        
        # ✅ CHANGE 5: Return negative MSE for maximization
        return -val_mse
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('-inf')

# Progress callback
def print_progress(study, trial):
    if trial.number % 25 == 0:
        # ✅ CHANGE 6: Show validation RMSE
        val_rmse = np.sqrt(-study.best_value)
        print(f"Completed {trial.number + 1} trials. Best validation RMSE: ${val_rmse:,.2f}")

print("\nStarting SVR hyperparameter optimization with Optuna...")
print("🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)")  # ✅ CHANGE 7
print("Using StandardScaler + SVR pipeline")
print("Testing 100 different parameter combinations...")
print("Parameter ranges:")
print("  C: 0.1-1000 (log scale)")
print("  epsilon: 0.01-1.0") 
print("  gamma: 'scale' or 'auto'")
print("  kernel: 'rbf', 'poly', or 'linear'")
print("  degree: 2-5 (for poly kernel only)")

print(f"\nUsing proper train/validation/test splits:")
print(f"  Training samples: {len(X_svr_train):,}")
print(f"  Validation samples: {len(X_svr_val):,}")
print(f"  Test samples: {len(X_svr_test):,}")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.NopPruner()
)

study.optimize(
    svr_objective, 
    n_trials=100,
    timeout=7200,
    callbacks=[print_progress],
    show_progress_bar=True,
    n_jobs=1
)

# ✅ CHANGE 8: Updated results display
best_val_rmse = np.sqrt(-study.best_value)
print(f"\n=== SVR Optuna Results (Optimized for Validation) ===")
print(f"Completed {len(study.trials)} out of 100 trials")
print("Best trial:")
print(f"  Trial number: {study.best_trial.number}")
print(f"  Value (neg_MSE): {study.best_trial.value}")
print(f"  Validation RMSE: ${best_val_rmse:,.2f}")
print("  Best params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

print("\nTraining final SVR model with best parameters...")

# Create best pipeline
best_svr_params = study.best_trial.params.copy()
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(**best_svr_params))
])

# ✅ CHANGE 9: Fit on training data only
best_pipeline.fit(X_svr_train, y_train)

# ✅ CHANGE 10: Get predictions for all three sets
train_predictions = best_pipeline.predict(X_svr_train)
val_predictions = best_pipeline.predict(X_svr_val)
test_predictions = best_pipeline.predict(X_svr_test)

# ✅ CHANGE 11: Calculate metrics for all three sets
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

train_mae = mean_absolute_error(y_train, train_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)

train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)
test_r2 = r2_score(y_test, test_predictions)

# ✅ CHANGE 12: Display performance across all sets
print(f"\n=== Model Performance Across All Sets ===")
print(f"Training Set:")
print(f"  RMSE: ${train_rmse:,.2f}")
print(f"  MAE: ${train_mae:,.2f}")
print(f"  R²: {train_r2:.4f}")

print(f"\nValidation Set:")
print(f"  RMSE: ${val_rmse:,.2f}")
print(f"  MAE: ${val_mae:,.2f}")
print(f"  R²: {val_r2:.4f}")

print(f"\nTest Set (UNSEEN DATA):")
print(f"  RMSE: ${test_rmse:,.2f}")
print(f"  MAE: ${test_mae:,.2f}")
print(f"  R²: {test_r2:.4f}")

# ✅ CHANGE 13: Check for overfitting
print(f"\n=== Overfitting Analysis ===")
if train_rmse < val_rmse * 0.8:
    print(f"⚠️  WARNING: Possible overfitting detected!")
    print(f"   Training RMSE ({train_rmse:,.2f}) much lower than validation RMSE ({val_rmse:,.2f})")
else:
    print(f"✅ Model generalizes well - training and validation performance similar")

if abs(val_rmse - test_rmse) / val_rmse < 0.1:
    print(f"✅ Validation set is good proxy for test performance")
else:
    print(f"⚠️  Large difference between validation and test performance")

# ✅ CHANGE 14: Create results with TEST set only (true unseen data)
results_df = X_svr_test.copy()
results_df['actual_price'] = y_test
results_df['svr_predicted_price'] = test_predictions
results_df['prediction_error'] = results_df['actual_price'] - results_df['svr_predicted_price']
results_df['absolute_error'] = abs(results_df['prediction_error'])

# Save to CSV
results_df.to_csv('/Users/clarencemarvin/Downloads/regularized2/svr_predictions.csv', index=False)

# Trial summary
failed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]
print(f"\nTrial Summary:")
print(f"  Total trials: {len(study.trials)}")
print(f"  Successful trials: {len(study.trials) - len(failed_trials)}")
print(f"  Failed trials: {len(failed_trials)}")

print(f"\nSVR Details:")
print(f"  Best kernel: {study.best_trial.params['kernel']}")
print(f"  Number of support vectors: {best_pipeline.named_steps['svr'].n_support_}")
print(f"  Support vector ratio: {sum(best_pipeline.named_steps['svr'].n_support_) / len(X_svr_train):.2%}")

print(f"\n🎯 FINAL RESULT: Test RMSE ${test_rmse:,.2f} on truly unseen data!")

[I 2025-11-11 12:55:09,060] A new study created in memory with name: no-name-1801bae4-64e8-4c1d-a4f2-899406896a52


=== SVR Data Preparation ===
Training data shape: (3693, 10)
Validation data shape: (1231, 10)
Test data shape: (1232, 10)
Features: ['bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary', 'district_encoded']

Starting SVR hyperparameter optimization with Optuna...
🎯 OPTIMIZING FOR VALIDATION PERFORMANCE (not training!)
Using StandardScaler + SVR pipeline
Testing 100 different parameter combinations...
Parameter ranges:
  C: 0.1-1000 (log scale)
  epsilon: 0.01-1.0
  gamma: 'scale' or 'auto'
  kernel: 'rbf', 'poly', or 'linear'
  degree: 2-5 (for poly kernel only)

Using proper train/validation/test splits:
  Training samples: 3,693
  Validation samples: 1,231
  Test samples: 1,232


Best trial: 0. Best value: -2.39496e+13:   1%|          | 1/100 [00:00<01:17,  1.28it/s, 0.78/7200 seconds]

[I 2025-11-11 12:55:09,845] Trial 0 finished with value: -23949648974474.086 and parameters: {'C': 1.3893207134677135, 'epsilon': 0.5088024139453727, 'gamma': 'auto', 'kernel': 'poly', 'degree': 2}. Best is trial 0 with value: -23949648974474.086.
Completed 1 trials. Best validation RMSE: $4,893,837.86


Best trial: 1. Best value: -2.39494e+13:   2%|▏         | 2/100 [00:01<01:28,  1.11it/s, 1.77/7200 seconds]

...

[I 2025-11-11 12:56:55,856] Trial 97 finished with value: -23346400497254.477 and parameters: {'C': 642.4832196180124, 'epsilon': 0.9980102488063735, 'gamma': 'scale', 'kernel': 'rbf'}. Best is trial 87 with value: -11734133847887.545.


Best trial: 87. Best value: -1.17341e+13:  99%|█████████▉| 99/100 [01:47<00:01,  1.05s/it, 107.86/7200 seconds]

[I 2025-11-11 12:56:56,925] Trial 98 finished with value: -18464906570278.965 and parameters: {'C': 244.81671144775592, 'epsilon': 0.9704091832180466, 'gamma': 'scale', 'kernel': 'linear'}. Best is trial 87 with value: -11734133847887.545.


Best trial: 87. Best value: -1.17341e+13: 100%|██████████| 100/100 [01:48<00:00,  1.09s/it, 108.86/7200 seconds]


[I 2025-11-11 12:56:57,918] Trial 99 finished with value: -23796188561852.707 and parameters: {'C': 6.66854593360925, 'epsilon': 0.9288872205180897, 'gamma': 'auto', 'kernel': 'linear'}. Best is trial 87 with value: -11734133847887.545.

=== SVR Optuna Results (Optimized for Validation) ===
Completed 100 out of 100 trials
Best trial:
  Trial number: 87
  Value (neg_MSE): -11734133847887.545
  Validation RMSE: $3,425,512.20
  Best params:
    C: 998.9389503962157
    epsilon: 0.9616645292151559
    gamma: scale
    kernel: linear

Training final SVR model with best parameters...

=== Model Performance Across All Sets ===
Training Set:
  RMSE: $3,807,574.61
  MAE: $2,112,581.10
  R²: 0.4451

Validation Set:
  RMSE: $3,425,512.20
  MAE: $2,132,129.29
  R²: 0.4882

Test Set (UNSEEN DATA):
  RMSE: $3,742,553.33
  MAE: $2,203,187.33
  R²: 0.4631

=== Overfitting Analysis ===
✅ Model generalizes well - training and validation performance similar
✅ Validation set is good proxy for test perform

In [66]:
import pandas as pd

print("=== Creating Consolidated DataFrame ===")

# Load all model predictions
print("Loading model prediction files...")
lgbm_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/lgb_predictions_regularized.csv')
xgb_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/xgb_predictions_regularized.csv')
cb_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/catboost_predictions_regularized.csv')
rf_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/rf_predictions_regularized.csv')
svr_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/svr_predictions.csv')

# ✅ Filter RF to test data only
rf_df = rf_df[rf_df['dataset'] == 'test'].reset_index(drop=True)
print(f"✅ RF filtered to test set only: {len(rf_df)} rows")

print(f"✅ All files loaded. Each contains {len(lgbm_df)} test samples")

# Start with features and actual prices from any model (they're identical)
# Drop the model-specific prediction columns and error columns
consolidated_df = lgbm_df.drop(['lgb_predicted_price', 'prediction_error', 'absolute_error'], axis=1).copy()

# Add all model predictions with corrected column names
consolidated_df['lgbm_predicted'] = lgbm_df['lgb_predicted_price']
consolidated_df['xgb_predicted'] = xgb_df['xgb_predicted_price'] 
consolidated_df['catboost_predicted'] = cb_df['catboost_predicted_price']
consolidated_df['rf_predicted'] = rf_df['rf_predicted_price']
consolidated_df['svr_predicted'] = svr_df['svr_predicted_price']

# Save consolidated results
consolidated_df.to_csv('/Users/clarencemarvin/Downloads/regularized/consolidated_predictions.csv', index=False)

print(f"\n✅ Consolidated dataframe created and saved!")
print(f"📁 File: /Users/clarencemarvin/Downloads/regularized/consolidated_predictions.csv")
print(f"\n📊 Dataframe info:")
print(f"  Shape: {consolidated_df.shape}")
print(f"  Columns: {list(consolidated_df.columns)}")
print(f"\n📋 Sample data:")
print(consolidated_df[['actual_price', 'lgbm_predicted', 'xgb_predicted', 'catboost_predicted', 'rf_predicted', 'svr_predicted']].head())

=== Creating Consolidated DataFrame ===
Loading model prediction files...
✅ RF filtered to test set only: 1232 rows
✅ All files loaded. Each contains 1232 test samples

✅ Consolidated dataframe created and saved!
📁 File: /Users/clarencemarvin/Downloads/regularized/consolidated_predictions.csv

📊 Dataframe info:
  Shape: (1232, 16)
  Columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary', 'actual_price', 'lgbm_predicted', 'xgb_predicted', 'catboost_predicted', 'rf_predicted', 'svr_predicted']

📋 Sample data:
   actual_price  lgbm_predicted  xgb_predicted  catboost_predicted  \
0       8700000    8.559522e+06      8258734.5        8.913412e+06   
1      23000000    2.050051e+07     21432890.0        1.995195e+07   
2       7700000    6.030042e+06      6002828.5        6.171043e+06   
3       5000000

In [70]:
import pandas as pd
import numpy as np

print("=== Simple Average Ensemble ===")

# Load the consolidated dataframe
consolidated_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/consolidated_predictions.csv')

print(f"Loaded consolidated data: {consolidated_df.shape}")

# Create simple average of all 5 models
consolidated_df['ensemble_simple_avg'] = (
    consolidated_df['lgbm_predicted'] + 
    consolidated_df['xgb_predicted'] + 
    consolidated_df['catboost_predicted'] + 
    consolidated_df['rf_predicted'] + 
    consolidated_df['svr_predicted']
) / 5

# Calculate prediction error
consolidated_df['ensemble_error'] = consolidated_df['actual_price'] - consolidated_df['ensemble_simple_avg']

# Calculate RMSE
ensemble_rmse = np.sqrt(np.mean(consolidated_df['ensemble_error']**2))

print(f"\n🎯 Simple Average Ensemble Results:")
print(f"  RMSE: ${ensemble_rmse:,.2f}")

# For comparison, show individual model RMSEs
print(f"\n📊 Individual Model RMSEs (for comparison):")
individual_models = ['lgbm_predicted', 'xgb_predicted', 'catboost_predicted', 'rf_predicted', 'svr_predicted']

for model in individual_models:
    error = consolidated_df['actual_price'] - consolidated_df[model]
    rmse = np.sqrt(np.mean(error**2))
    model_name = model.replace('_predicted', '').upper()
    print(f"  {model_name}: ${rmse:,.2f}")

print(f"\n📈 Improvement Analysis:")
best_individual = min([np.sqrt(np.mean((consolidated_df['actual_price'] - consolidated_df[model])**2)) for model in individual_models])
print(f"  Best individual model RMSE: ${best_individual:,.2f}")
print(f"  Ensemble RMSE: ${ensemble_rmse:,.2f}")
improvement = best_individual - ensemble_rmse
print(f"  Improvement: ${improvement:,.2f} ({improvement/best_individual*100:.2f}%)")

# Show sample predictions
print(f"\n🔍 Sample Ensemble Predictions:")
sample_cols = ['actual_price', 'xgb_predicted', 'ensemble_simple_avg', 'ensemble_error']
print(consolidated_df[sample_cols].head(10))

=== Simple Average Ensemble ===
Loaded consolidated data: (1232, 16)

🎯 Simple Average Ensemble Results:
  RMSE: $2,019,293.46

📊 Individual Model RMSEs (for comparison):
  LGBM: $1,984,085.78
  XGB: $2,006,796.98
  CATBOOST: $2,016,677.70
  RF: $2,030,651.75
  SVR: $3,745,631.54

📈 Improvement Analysis:
  Best individual model RMSE: $1,984,085.78
  Ensemble RMSE: $2,019,293.46
  Improvement: $-35,207.68 (-1.77%)

🔍 Sample Ensemble Predictions:
   actual_price  xgb_predicted  ensemble_simple_avg  ensemble_error
0       8700000      8258734.5         8.604547e+06    9.545264e+04
1      23000000     21432890.0         1.823278e+07    4.767224e+06
2       7700000      6002828.5         6.155932e+06    1.544068e+06
3       5000000      4592087.0         4.696774e+06    3.032262e+05
4       3800000      5305176.0         5.092641e+06   -1.292641e+06
5       9900000     11165261.0         1.130909e+07   -1.409086e+06
6      23000000     18796276.0         1.773522e+07    5.264783e+06
7      

In [71]:
import numpy as np
import pandas as pd

df = consolidated_df.copy()

# Ensure numeric
pred_cols = [
    'lgbm_predicted',
    'xgb_predicted',
    'rf_predicted',
    'svr_predicted',
    'ensemble_simple_avg'
]

df = df[['actual_price'] + pred_cols].apply(pd.to_numeric, errors='coerce')

# Optionally drop rows with missing targets or predictions
df = df.dropna(subset=['actual_price'] + pred_cols)

y = df['actual_price'].to_numpy(dtype=float)
y_mean = y.mean()

def r2(y, yhat):
    ss_res = np.sum((y - yhat) ** 2)
    ss_tot = np.sum((y - y_mean) ** 2)
    return 1 - ss_res / ss_tot if ss_tot != 0 else np.nan

def mape(y, yhat):
    # Filter out zero targets to avoid division by zero
    mask = y != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100

def mdape(y, yhat):
    mask = y != 0
    if not np.any(mask):
        return np.nan
    return np.median(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100

results = []
for col in pred_cols:
    yhat = df[col].to_numpy(dtype=float)
    results.append({
        'model': col,
        'R2': r2(y, yhat),
        'MAPE_%': mape(y, yhat),
        'MdAPE_%': mdape(y, yhat)
    })

metrics_df = pd.DataFrame(results).sort_values('R2', ascending=False).reset_index(drop=True)
print(metrics_df)

                 model        R2     MAPE_%    MdAPE_%
0       lgbm_predicted  0.849107  12.141942   8.438442
1        xgb_predicted  0.845633  12.316936   8.573524
2  ensemble_simple_avg  0.843705  12.898161   9.089698
3         rf_predicted  0.841942  13.869890   9.346203
4        svr_predicted  0.462230  23.103064  17.219843


In [3]:
import pandas as pd
import numpy as np

print("=== Simple Average Ensemble (4 Models - Excluding SVR) ===")

# Load the consolidated dataframe
consolidated_df = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/consolidated_predictions.csv')

print(f"Loaded consolidated data: {consolidated_df.shape}")

# Create simple average of 4 models (excluding SVR)
consolidated_df['ensemble_avg_4models'] = (
    consolidated_df['lgbm_predicted'] + 
    consolidated_df['xgb_predicted'] + 
    consolidated_df['catboost_predicted'] + 
    consolidated_df['rf_predicted']
) / 4

# Calculate prediction error
consolidated_df['ensemble_error'] = consolidated_df['actual_price'] - consolidated_df['ensemble_avg_4models']

# Calculate RMSE
ensemble_rmse = np.sqrt(np.mean(consolidated_df['ensemble_error']**2))

print(f"\n🎯 4-Model Average Ensemble Results:")
print(f"  RMSE: ${ensemble_rmse:,.2f}")

# For comparison, show individual model RMSEs (excluding SVR)
print(f"\n📊 Individual Model RMSEs (for comparison):")
individual_models = ['lgbm_predicted', 'xgb_predicted', 'catboost_predicted', 'rf_predicted']

for model in individual_models:
    error = consolidated_df['actual_price'] - consolidated_df[model]
    rmse = np.sqrt(np.mean(error**2))
    model_name = model.replace('_predicted', '').upper()
    print(f"  {model_name}: ${rmse:,.2f}")

print(f"\n📈 Improvement Analysis:")
best_individual = min([np.sqrt(np.mean((consolidated_df['actual_price'] - consolidated_df[model])**2)) for model in individual_models])
print(f"  Best individual model RMSE: ${best_individual:,.2f}")
print(f"  4-Model Ensemble RMSE: ${ensemble_rmse:,.2f}")
improvement = best_individual - ensemble_rmse
print(f"  Improvement: ${improvement:,.2f} ({improvement/best_individual*100:.2f}%)")

# Show sample predictions
print(f"\n🔍 Sample Ensemble Predictions:")
sample_cols = ['actual_price', 'xgb_predicted', 'ensemble_avg_4models', 'ensemble_error']
print(consolidated_df[sample_cols].head(10))

=== Simple Average Ensemble (4 Models - Excluding SVR) ===
Loaded consolidated data: (1232, 17)

🎯 4-Model Average Ensemble Results:
  RMSE: $1,962,236.31

📊 Individual Model RMSEs (for comparison):
  LGBM: $2,025,296.28
  XGB: $2,010,380.24
  CATBOOST: $2,016,677.70
  RF: $2,060,934.25

📈 Improvement Analysis:
  Best individual model RMSE: $2,010,380.24
  4-Model Ensemble RMSE: $1,962,236.31
  Improvement: $48,143.92 (2.39%)

🔍 Sample Ensemble Predictions:
   actual_price  xgb_predicted  ensemble_avg_4models  ensemble_error
0       8700000      8597520.0          8.513770e+06    1.862300e+05
1      23000000     20328726.0          1.978787e+07    3.212131e+06
2       7700000      6100254.5          6.097540e+06    1.602460e+06
3       5000000      4430923.5          4.425412e+06    5.745879e+05
4       3800000      5217821.5          4.877022e+06   -1.077022e+06
5       9900000     11267624.0          1.147948e+07   -1.579478e+06
6      23000000     17939796.0          1.886848e+07   

In [26]:
import pandas as pd

# Load all the "all_data" CSV files from each model
lgb_all = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/lgb_predictions_all_data.csv')
xgb_all = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/xgb_predictions_all_data.csv')
catboost_all = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/catboost_predictions_all_data.csv')
rf_all = pd.read_csv('/Users/clarencemarvin/Downloads/regularized/rf_predictions_regularized.csv')

# Start with LightGBM as the base (has all features + actual_price + dataset)
all_consolidated_df = lgb_all[['district', 'bedroom_count', 'property_age', 'saleable_area',
       'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m',
       'category_Education_within_2000m', 'category_Medical_within_2000m',
       'category_Public_Market_within_1000m', 'pet_policy_binary', 'dataset',
       'actual_price', 'lgb_predicted_price', 'prediction_error',
       'absolute_error']].copy()

# Add prediction columns from each model
all_consolidated_df['lgbm_predicted'] = lgb_all['lgb_predicted_price']
all_consolidated_df['xgb_predicted'] = xgb_all['xgb_predicted_price']
all_consolidated_df['catboost_predicted'] = catboost_all['catboost_predicted_price']
all_consolidated_df['rf_predicted'] = rf_all['rf_predicted_price']

# Save the consolidated file
all_consolidated_df.to_csv('/Users/clarencemarvin/Downloads/regularized/all_consolidated_predictions.csv', index=False)

print(f"✅ Created consolidated DataFrame with {len(all_consolidated_df):,} rows")
print(f"Columns: {list(all_consolidated_df.columns)}")
print(f"\nDataset breakdown:")
print(all_consolidated_df['dataset'].value_counts())

✅ Created consolidated DataFrame with 6,156 rows
Columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary', 'dataset', 'actual_price', 'lgb_predicted_price', 'prediction_error', 'absolute_error', 'lgbm_predicted', 'xgb_predicted', 'catboost_predicted', 'rf_predicted']

Dataset breakdown:
dataset
train         3693
test          1232
validation    1231
Name: count, dtype: int64


In [28]:
all_consolidated_df.head()

,district,bedroom_count,property_age,saleable_area,travel_time_to_cbd,walking_time_to_mtr,total_poi_within_1000m,category_Education_within_2000m,category_Medical_within_2000m,category_Public_Market_within_1000m,pet_policy_binary,dataset,actual_price,lgb_predicted_price,prediction_error,absolute_error,lgbm_predicted,xgb_predicted,catboost_predicted,rf_predicted
0,Ho Man Tin,3,36,978,26,16,838,484,64,3,1,train,12500000,1.237105e+07,1.289491e+05,1.289491e+05,1.237105e+07,12540288.0,1.422219e+07,1.249192e+07
1,Happy Valley,3,40,750,21,9,797,164,27,5,1,train,13000000,1.328240e+07,-2.823989e+05,2.823989e+05,1.328240e+07,12856448.0,1.239582e+07,1.284752e+07
2,Tai Wai,1,37,282,45,18,391,115,6,10,1,train,3300000,3.801891e+06,-5.018914e+05,5.018914e+05,3.801891e+06,3411503.0,4.082155e+06,3.662088e+06
3,Sai Ying Pun,3,48,737,14,9,760,145,25,7,1,train,10680000,1.065484e+07,2.516439e+04,2.516439e+04,1.065484e+07,10797545.0,1.185482e+07,1.125219e+07
4,Tsing Yi,2,39,381,41,19,484,112,5,6,1,train,2900000,4.127934e+06,-1.227934e+06,1.227934e+06,4.127934e+06,4054065.0,4.219507e+06,3.968541e+06


In [31]:
df = all_consolidated_df.copy()

df['lgbm_error'] = abs(df['lgbm_predicted'] - df['actual_price'])
df['xgb_error'] = abs(df['xgb_predicted'] - df['actual_price'])
df['catboost_error'] = abs(df['catboost_predicted'] - df['actual_price'])
df['rf_error'] = abs(df['rf_predicted'] - df['actual_price'])

error_columns = ['lgbm_error', 'xgb_error', 'catboost_error', 'rf_error']
best_model_idx = df[error_columns].idxmin(axis=1)

model_mapping = {
    'lgbm_error': 'LGBM',
    'xgb_error': 'XGB',
    'catboost_error': 'CatBoost',
    'rf_error': 'RF'
}
df['best_model'] = best_model_idx.map(model_mapping)

df['dataset'].value_counts()

dataset
train         3693
test          1232
validation    1231
Name: count, dtype: int64

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ============================================================================
# STEP 1: Define Feature and Target Columns
# ============================================================================

print("="*80)
print("STEP 1: Defining Features and Target")
print("="*80)

# Define which columns to use for meta-learning (only original features - NO LEAKAGE!)
FEATURE_COLS = ['district', 'bedroom_count', 'property_age', 'saleable_area',
       'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m',
       'category_Education_within_2000m', 'category_Medical_within_2000m',
       'category_Public_Market_within_1000m', 'pet_policy_binary']

# Target columns
PRICE_TARGET = 'actual_price'  # For base learners (regression)
META_TARGET = 'best_model'  # For meta-classifiers (which model is best)

print(f"Feature columns: {FEATURE_COLS}")
print(f"Price target: {PRICE_TARGET}")
print(f"Meta target: {META_TARGET}")

# ============================================================================
# STEP 2: Extract Features and Target from your dataframe
# ============================================================================

print("\n" + "="*80)
print("STEP 2: Extracting Data")
print("="*80)

# Assuming your dataframe is called 'df'
df = all_consolidated_df.copy()
X = df[FEATURE_COLS].copy()
y_price = df[PRICE_TARGET].copy()
y_meta = df[META_TARGET].copy() if META_TARGET in df.columns else None

print(f"\nOriginal data shape: {X.shape}")
print(f"Target (price) shape: {y_price.shape}")
if y_meta is not None:
    print(f"Target (best_model) shape: {y_meta.shape}")
    print(f"Best model distribution:\n{y_meta.value_counts()}")

# Check data types
print(f"\nFeature data types BEFORE conversion:")
print(X.dtypes)

# CRITICAL: Convert object columns to category dtype for tree models
print("\nConverting object columns to category...")
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        print(f"  Converted {col}: object → category")

print(f"\nFeature data types AFTER conversion:")
print(X.dtypes)

# ============================================================================
# STEP 3: Split Data into Train (64%), Validation (16%), Test (20%)
# ============================================================================

print("\n" + "="*80)
print("STEP 3: Splitting Data")
print("="*80)

# First split: 80% train+val, 20% test
X_temp, X_test, y_price_temp, y_test_price = train_test_split(
    X, y_price, test_size=0.20, random_state=42
)

if y_meta is not None:
    _, y_test_meta = train_test_split(
        y_meta, test_size=0.20, random_state=42
    )

# Second split: 80% train, 20% val (of the 80%)
X_train, X_val, y_train_price, y_val_price = train_test_split(
    X_temp, y_price_temp, test_size=0.20, random_state=42
)

if y_meta is not None:
    y_meta_temp = y_meta.iloc[X_temp.index]
    _, y_val_meta = train_test_split(
        y_meta_temp, test_size=0.20, random_state=42
    )

print(f"\nTrain set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

# Rename for clarity
y_train = y_train_price
y_val = y_val_price
y_test = y_test_price

# ============================================================================
# STEP 4: Identify Categorical Columns
# ============================================================================

print("\n" + "="*80)
print("STEP 4: Identifying Categorical Columns")
print("="*80)

# Identify categorical columns (now they should all be 'category' dtype)
cat_columns = X_train.select_dtypes(include=['category']).columns.tolist()
print(f"\nCategorical columns detected: {cat_columns}")

# Store for later use
CAT_FEATURES = cat_columns.copy()

# ============================================================================
# STEP 5: Create Dataset Versions for Each Algorithm
# ============================================================================

print("\n" + "="*80)
print("STEP 5: Creating Algorithm-Specific Datasets")
print("="*80)

# ----------------------------------------------------------------------------
# 5.1 For LightGBM - Keep original (handles categorical natively)
# ----------------------------------------------------------------------------
print("\n[1/4] LightGBM datasets - keeping original with categorical")
# X_train, X_val, X_test already defined above - no changes needed
print(f"  Train: {X_train.shape}")
print(f"  Val: {X_val.shape}")
print(f"  Test: {X_test.shape}")
print(f"  District dtype: {X_train['district'].dtype}")

# ----------------------------------------------------------------------------
# 5.2 For XGBoost - Keep original (already category dtype)
# ----------------------------------------------------------------------------
print("\n[2/4] XGBoost datasets - keeping original with categorical dtype")
X_train_xgb = X_train.copy()
X_val_xgb = X_val.copy()
X_test_xgb = X_test.copy()

print(f"  Train: {X_train_xgb.shape}, district dtype: {X_train_xgb['district'].dtype}")
print(f"  Val: {X_val_xgb.shape}")
print(f"  Test: {X_test_xgb.shape}")
print(f"  ⚠️  Remember to use: enable_categorical=True, tree_method='hist'")

# ----------------------------------------------------------------------------
# 5.3 For CatBoost - Keep original (handles categorical natively)
# ----------------------------------------------------------------------------
print("\n[3/4] CatBoost datasets - keeping original with categorical")
X_train_cb = X_train.copy()
X_val_cb = X_val.copy()
X_test_cb = X_test.copy()

print(f"  Train: {X_train_cb.shape}")
print(f"  Val: {X_val_cb.shape}")
print(f"  Test: {X_test_cb.shape}")
print(f"  Categorical features: {CAT_FEATURES}")

# ----------------------------------------------------------------------------
# 5.4 For Random Forest - Encode categorical columns
# ----------------------------------------------------------------------------
print("\n[4/4] Random Forest datasets - encoding categorical")
X_train_rf = X_train.copy()
X_val_rf = X_val.copy()
X_test_rf = X_test.copy()

# Encode each categorical column
label_encoders = {}
for col in cat_columns:
    if col in X_train_rf.columns:
        le = LabelEncoder()
        
        # Fit on all unique values from train, val, and test
        all_vals = pd.concat([
            X_train_rf[col].astype(str), 
            X_val_rf[col].astype(str), 
            X_test_rf[col].astype(str)
        ]).unique()
        le.fit(all_vals)
        
        # Transform
        X_train_rf[col] = le.transform(X_train_rf[col].astype(str))
        X_val_rf[col] = le.transform(X_val_rf[col].astype(str))
        X_test_rf[col] = le.transform(X_test_rf[col].astype(str))
        
        # Store encoder
        label_encoders[col] = le

print(f"  Train: {X_train_rf.shape}")
print(f"  Val: {X_val_rf.shape}")
print(f"  Test: {X_test_rf.shape}")
print(f"  Encoded columns: {list(label_encoders.keys())}")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("SUMMARY - All Datasets Ready!")
print("="*80)

print("\n📊 Dataset Splits:")
print(f"  Train: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Val:   {len(X_val):,} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test:  {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")

print("\n🎯 Target Variables:")
print(f"  y_train: {y_train.shape} (price)")
print(f"  y_val: {y_val.shape} (price)")
print(f"  y_test: {y_test.shape} (price)")
if y_meta is not None:
    print(f"  y_val_meta: {y_val_meta.shape} (best model labels)")
    print(f"  y_test_meta: {y_test_meta.shape} (best model labels)")

print("\n🤖 Algorithm-Specific Datasets:")
print("  ✓ LightGBM: X_train, X_val, X_test (category dtype)")
print("  ✓ XGBoost:  X_train_xgb, X_val_xgb, X_test_xgb (category dtype)")
print("             Use: enable_categorical=True, tree_method='hist'")
print("  ✓ CatBoost: X_train_cb, X_val_cb, X_test_cb (category dtype)")
print("  ✓ Random Forest: X_train_rf, X_val_rf, X_test_rf (label encoded)")

print("\n📝 Categorical Features:")
print(f"  CAT_FEATURES = {CAT_FEATURES}")

print("\n✅ Data splitting complete! Ready for model training.")
print("="*80)

#keknya boleh hapus

STEP 1: Defining Features and Target
Feature columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary']
Price target: actual_price
Meta target: best_model

STEP 2: Extracting Data

Original data shape: (1232, 11)
Target (price) shape: (1232,)
Target (best_model) shape: (1232,)
Best model distribution:
best_model
RF          333
CatBoost    314
XGB         310
LGBM        275
Name: count, dtype: int64

Feature data types BEFORE conversion:
district                               object
bedroom_count                           int64
property_age                            int64
saleable_area                           int64
travel_time_to_cbd                      int64
walking_time_to_mtr                     int64
total_poi_within_1000m                  int64
category_Education_with

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

FEATURE_COLS = ['district', 'bedroom_count', 'property_age', 'saleable_area',
                'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m',
                'category_Education_within_2000m', 'category_Medical_within_2000m',
                'category_Public_Market_within_1000m', 'pet_policy_binary']
TARGET_COL = 'best_model'

X = df[FEATURE_COLS].copy()
y = df[TARGET_COL].copy()
dataset_split = df['dataset'].copy()

X['district'] = X['district'].astype('category')

X_train = X[dataset_split == 'train'].copy()
X_val = X[dataset_split == 'validation'].copy()
X_test = X[dataset_split == 'test'].copy()

y_train = y[dataset_split == 'train'].copy()
y_val = y[dataset_split == 'validation'].copy()
y_test = y[dataset_split == 'test'].copy()

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

CAT_FEATURES = ['district']

# ============================================================================
# Create algorithm-specific datasets
# ============================================================================

X_train_lgbm = X_train.copy()
X_val_lgbm = X_val.copy()
X_test_lgbm = X_test.copy()

X_train_xgb = X_train.copy()
X_val_xgb = X_val.copy()
X_test_xgb = X_test.copy()
X_train_xgb['district'] = X_train_xgb['district'].cat.codes
X_val_xgb['district'] = X_val_xgb['district'].cat.codes
X_test_xgb['district'] = X_test_xgb['district'].cat.codes

X_train_cb = X_train.copy()
X_val_cb = X_val.copy()
X_test_cb = X_test.copy()

X_train_rf = X_train.copy()
X_val_rf = X_val.copy()
X_test_rf = X_test.copy()

le_district = LabelEncoder()
all_districts = pd.concat([X_train_rf['district'], X_val_rf['district'], X_test_rf['district']]).astype(str)
le_district.fit(all_districts)

X_train_rf['district'] = le_district.transform(X_train_rf['district'].astype(str))
X_val_rf['district'] = le_district.transform(X_val_rf['district'].astype(str))
X_test_rf['district'] = le_district.transform(X_test_rf['district'].astype(str))

# ============================================================================
# STEP 2: Train Meta-Classifiers
# ============================================================================

print("\n" + "="*80)
print("STEP 2: Training Meta-Classifiers")
print("="*80)

meta_predictions_test = {}

# ----------------------------------------------------------------------------
# 2.1 LightGBM Meta-Classifier
# ----------------------------------------------------------------------------
print("\n[1/4] Training LightGBM meta-classifier...")

def optimize_lgbm_meta(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 150),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 15, 35),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
        'max_depth': trial.suggest_int('max_depth', 5, 7),
        'reg_alpha': trial.suggest_float('reg_alpha', 2.5, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 2.5, 5.0),
        'objective': 'multiclass',
        'num_class': len(label_encoder.classes_),
        'random_state': 42,
        'verbosity': -1
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_lgbm, y_train_encoded)
    y_pred = model.predict(X_val_lgbm)
    return accuracy_score(y_val_encoded, y_pred)

study_lgbm_meta = optuna.create_study(direction='maximize')
study_lgbm_meta.optimize(optimize_lgbm_meta, n_trials=50, show_progress_bar=True)
print(f"Best LGBM Meta Validation Accuracy: {study_lgbm_meta.best_value:.4f}")

best_lgbm_meta = lgb.LGBMClassifier(**study_lgbm_meta.best_params)
best_lgbm_meta.fit(X_train_lgbm, y_train_encoded)
meta_predictions_test['LGBM_predict'] = label_encoder.inverse_transform(best_lgbm_meta.predict(X_test_lgbm))

with open('/Users/clarencemarvin/Downloads/regularized/lgbm_meta_model.pkl', 'wb') as f:
    pickle.dump(best_lgbm_meta, f)

# ----------------------------------------------------------------------------
# 2.2 XGBoost Meta-Classifier
# ----------------------------------------------------------------------------
print("\n[2/4] Training XGBoost meta-classifier...")

def optimize_xgb_meta(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.85),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 2.5, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 2.5, 5.0),
        'random_state': 42,
        'n_jobs': -1
    }
    model = xgb.XGBClassifier(**params)
    model.fit(X_train_xgb, y_train_encoded)
    y_pred = model.predict(X_val_xgb)
    return accuracy_score(y_val_encoded, y_pred)

study_xgb_meta = optuna.create_study(direction='maximize')
study_xgb_meta.optimize(optimize_xgb_meta, n_trials=50, show_progress_bar=True)
print(f"Best XGB Meta Validation Accuracy: {study_xgb_meta.best_value:.4f}")

best_xgb_meta = xgb.XGBClassifier(**study_xgb_meta.best_params)
best_xgb_meta.fit(X_train_xgb, y_train_encoded)
meta_predictions_test['XGB_predict'] = label_encoder.inverse_transform(best_xgb_meta.predict(X_test_xgb))

with open('/Users/clarencemarvin/Downloads/regularized/xgb_meta_model.pkl', 'wb') as f:
    pickle.dump(best_xgb_meta, f)

# ----------------------------------------------------------------------------
# 2.3 CatBoost Meta-Classifier
# ----------------------------------------------------------------------------
print("\n[3/4] Training CatBoost meta-classifier...")

def optimize_cb_meta(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.15),
        'iterations': trial.suggest_int('iterations', 100, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 2.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'loss_function': 'MultiClass',
        'random_state': 42,
        'verbose': False,
        'cat_features': CAT_FEATURES
    }
    model = CatBoostClassifier(**params)
    model.fit(X_train_cb, y_train_encoded, cat_features=CAT_FEATURES, verbose=False)
    y_pred = model.predict(X_val_cb)
    return accuracy_score(y_val_encoded, y_pred.flatten())

study_cb_meta = optuna.create_study(direction='maximize')
study_cb_meta.optimize(optimize_cb_meta, n_trials=50, show_progress_bar=True)
print(f"Best CatBoost Meta Validation Accuracy: {study_cb_meta.best_value:.4f}")

best_cb_meta = CatBoostClassifier(**study_cb_meta.best_params)
best_cb_meta.fit(X_train_cb, y_train_encoded, cat_features=CAT_FEATURES, verbose=False)
meta_predictions_test['CB_predict'] = label_encoder.inverse_transform(best_cb_meta.predict(X_test_cb).flatten())

with open('/Users/clarencemarvin/Downloads/regularized/catboost_meta_model.pkl', 'wb') as f:
    pickle.dump(best_cb_meta, f)

# ----------------------------------------------------------------------------
# 2.4 Random Forest Meta-Classifier
# ----------------------------------------------------------------------------
print("\n[4/4] Training Random Forest meta-classifier...")

def optimize_rf_meta(trial):
    max_depth_choice = trial.suggest_categorical('max_depth_type', ['int', 'none'])
    max_depth = trial.suggest_int('max_depth', 3, 12) if max_depth_choice == 'int' else None
    
    max_leaf_nodes_choice = trial.suggest_categorical('max_leaf_nodes_type', ['int', 'none'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 10, 100) if max_leaf_nodes_choice == 'int' else None
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_features': trial.suggest_categorical('max_features', [0.3, 0.5, 0.7]),
        'max_depth': max_depth,
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 5, 20),
        'max_leaf_nodes': max_leaf_nodes,
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'random_state': 42,
        'n_jobs': -1
    }
    model = RandomForestClassifier(**params)
    model.fit(X_train_rf, y_train_encoded)
    y_pred = model.predict(X_val_rf)
    return accuracy_score(y_val_encoded, y_pred)

study_rf_meta = optuna.create_study(direction='maximize')
study_rf_meta.optimize(optimize_rf_meta, n_trials=50, show_progress_bar=True)
print(f"Best RF Meta Validation Accuracy: {study_rf_meta.best_value:.4f}")

rf_meta_params = {k: v for k, v in study_rf_meta.best_params.items() 
                  if k not in ['max_depth_type', 'max_leaf_nodes_type']}
rf_meta_params['random_state'] = 42
rf_meta_params['n_jobs'] = -1

best_rf_meta = RandomForestClassifier(**rf_meta_params)
best_rf_meta.fit(X_train_rf, y_train_encoded)
meta_predictions_test['RF_predict'] = label_encoder.inverse_transform(best_rf_meta.predict(X_test_rf))

with open('/Users/clarencemarvin/Downloads/regularized/rf_meta_model.pkl', 'wb') as f:
    pickle.dump(best_rf_meta, f)

# Save label encoder
with open('/Users/clarencemarvin/Downloads/regularized/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# ============================================================================
# STEP 3: Evaluate and Save Results
# ============================================================================

print("\n" + "="*80)
print("STEP 3: Evaluation and Results")
print("="*80)

# Create output dataframe for ALL data (train, validation, test)
output_df = X.copy()
output_df['dataset'] = dataset_split

# Get predictions for each split
meta_predictions_all = {}

# Train predictions
meta_predictions_all['LGBM_train'] = label_encoder.inverse_transform(best_lgbm_meta.predict(X_train_lgbm))
meta_predictions_all['XGB_train'] = label_encoder.inverse_transform(best_xgb_meta.predict(X_train_xgb))
meta_predictions_all['CB_train'] = label_encoder.inverse_transform(best_cb_meta.predict(X_train_cb).flatten())
meta_predictions_all['RF_train'] = label_encoder.inverse_transform(best_rf_meta.predict(X_train_rf))

# Validation predictions
meta_predictions_all['LGBM_val'] = label_encoder.inverse_transform(best_lgbm_meta.predict(X_val_lgbm))
meta_predictions_all['XGB_val'] = label_encoder.inverse_transform(best_xgb_meta.predict(X_val_xgb))
meta_predictions_all['CB_val'] = label_encoder.inverse_transform(best_cb_meta.predict(X_val_cb).flatten())
meta_predictions_all['RF_val'] = label_encoder.inverse_transform(best_rf_meta.predict(X_val_rf))

# Test predictions
meta_predictions_all['LGBM_test'] = label_encoder.inverse_transform(best_lgbm_meta.predict(X_test_lgbm))
meta_predictions_all['XGB_test'] = label_encoder.inverse_transform(best_xgb_meta.predict(X_test_xgb))
meta_predictions_all['CB_test'] = label_encoder.inverse_transform(best_cb_meta.predict(X_test_cb).flatten())
meta_predictions_all['RF_test'] = label_encoder.inverse_transform(best_rf_meta.predict(X_test_rf))

# Combine all predictions back into output_df
output_df['LGBM_predict'] = np.nan
output_df['XGB_predict'] = np.nan
output_df['CB_predict'] = np.nan
output_df['RF_predict'] = np.nan

output_df.loc[dataset_split == 'train', 'LGBM_predict'] = meta_predictions_all['LGBM_train']
output_df.loc[dataset_split == 'train', 'XGB_predict'] = meta_predictions_all['XGB_train']
output_df.loc[dataset_split == 'train', 'CB_predict'] = meta_predictions_all['CB_train']
output_df.loc[dataset_split == 'train', 'RF_predict'] = meta_predictions_all['RF_train']

output_df.loc[dataset_split == 'validation', 'LGBM_predict'] = meta_predictions_all['LGBM_val']
output_df.loc[dataset_split == 'validation', 'XGB_predict'] = meta_predictions_all['XGB_val']
output_df.loc[dataset_split == 'validation', 'CB_predict'] = meta_predictions_all['CB_val']
output_df.loc[dataset_split == 'validation', 'RF_predict'] = meta_predictions_all['RF_val']

output_df.loc[dataset_split == 'test', 'LGBM_predict'] = meta_predictions_all['LGBM_test']
output_df.loc[dataset_split == 'test', 'XGB_predict'] = meta_predictions_all['XGB_test']
output_df.loc[dataset_split == 'test', 'CB_predict'] = meta_predictions_all['CB_test']
output_df.loc[dataset_split == 'test', 'RF_predict'] = meta_predictions_all['RF_test']

# Save
save_path = '/Users/clarencemarvin/Downloads/regularized/meta_classifier_predictions.csv'
output_df.to_csv(save_path, index=False)

print(f"Output shape: {output_df.shape}")
print(f"\nDataset distribution:\n{output_df['dataset'].value_counts()}")

print(f"\n✅ Saved to '{save_path}'")
print(f"\nOutput shape: {output_df.shape}")
print(f"Columns: {list(output_df.columns)}")

print("\nFirst 10 rows:")
print(output_df.head(10))

# Calculate accuracies
print("\n" + "="*80)
print("META-CLASSIFIER ACCURACIES")
print("="*80)

print("\n📊 TRUE Best Model Distribution on Test Set:")
print(y_test.value_counts())

accuracies = {}
accuracies['LGBM_meta'] = accuracy_score(y_test, meta_predictions_test['LGBM_predict'])
accuracies['XGB_meta'] = accuracy_score(y_test, meta_predictions_test['XGB_predict'])
accuracies['CB_meta'] = accuracy_score(y_test, meta_predictions_test['CB_predict'])
accuracies['RF_meta'] = accuracy_score(y_test, meta_predictions_test['RF_predict'])

print("\n🎯 Validation Set Performance:")
print(f"  LGBM Meta:     {study_lgbm_meta.best_value:.4f} ({study_lgbm_meta.best_value*100:.2f}%)")
print(f"  XGB Meta:      {study_xgb_meta.best_value:.4f} ({study_xgb_meta.best_value*100:.2f}%)")
print(f"  CatBoost Meta: {study_cb_meta.best_value:.4f} ({study_cb_meta.best_value*100:.2f}%)")
print(f"  RF Meta:       {study_rf_meta.best_value:.4f} ({study_rf_meta.best_value*100:.2f}%)")

print("\n🎯 Test Set Performance:")
for meta_name, acc in accuracies.items():
    print(f"  {meta_name:15s}: {acc:.4f} ({acc*100:.2f}%)")

best_meta = max(accuracies, key=accuracies.get)
print(f"\n🏆 Best Meta-Classifier: {best_meta} with {accuracies[best_meta]*100:.2f}% test accuracy")

# Detailed classification reports
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORTS")
print("="*80)

for name, predictions in [('LGBM', meta_predictions_test['LGBM_predict']),
                          ('XGB', meta_predictions_test['XGB_predict']),
                          ('CatBoost', meta_predictions_test['CB_predict']),
                          ('RF', meta_predictions_test['RF_predict'])]:
    print(f"\n{name} Meta-Classifier:")
    print(classification_report(y_test, predictions))

print("\n✅ All done!")
print("="*80)

[I 2025-11-12 14:18:30,392] A new study created in memory with name: no-name-d042bc7d-c9d2-4153-9956-15b39c95b588


Train: (3693, 11), Val: (1231, 11), Test: (1232, 11)

STEP 2: Training Meta-Classifiers

[1/4] Training LightGBM meta-classifier...


Best trial: 0. Best value: 0.330626:   2%|▏         | 1/50 [00:08<06:34,  8.06s/it]

[I 2025-11-12 14:18:38,465] Trial 0 finished with value: 0.330625507717303 and parameters: {'num_leaves': 56, 'min_data_in_leaf': 30, 'feature_fraction': 0.8008147416462852, 'n_estimators': 285, 'learning_rate': 0.16645755596285394, 'max_depth': 6, 'reg_alpha': 2.6305441857389473, 'reg_lambda': 4.872331305781033}. Best is trial 0 with value: 0.330625507717303.


Best trial: 0. Best value: 0.330626:   4%|▍         | 2/50 [00:13<05:08,  6.43s/it]

...

[I 2025-11-12 14:27:15,592] Trial 47 finished with value: 0.30381803411860275 and parameters: {'max_depth_type': 'none', 'max_leaf_nodes_type': 'none', 'n_estimators': 121, 'max_features': 0.5, 'min_samples_split': 19, 'min_samples_leaf': 11, 'bootstrap': False}. Best is trial 13 with value: 0.3241267262388302.


Best trial: 13. Best value: 0.324127:  98%|█████████▊| 49/50 [00:27<00:00,  1.59it/s]

[I 2025-11-12 14:27:16,143] Trial 48 finished with value: 0.3151909017059301 and parameters: {'max_depth_type': 'none', 'max_leaf_nodes_type': 'none', 'n_estimators': 100, 'max_features': 0.5, 'min_samples_split': 17, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 13 with value: 0.3241267262388302.


Best trial: 13. Best value: 0.324127: 100%|██████████| 50/50 [00:27<00:00,  1.80it/s]


[I 2025-11-12 14:27:16,667] Trial 49 finished with value: 0.31762794476035744 and parameters: {'max_depth_type': 'none', 'max_leaf_nodes_type': 'none', 'n_estimators': 86, 'max_features': 0.5, 'min_samples_split': 17, 'min_samples_leaf': 7, 'bootstrap': False}. Best is trial 13 with value: 0.3241267262388302.
Best RF Meta Validation Accuracy: 0.3241

STEP 3: Evaluation and Results
Output shape: (6156, 16)

Dataset distribution:
dataset
train         3693
test          1232
validation    1231
Name: count, dtype: int64

✅ Saved to '/Users/clarencemarvin/Downloads/regularized/meta_classifier_predictions.csv'

Output shape: (6156, 16)
Columns: ['district', 'bedroom_count', 'property_age', 'saleable_area', 'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m', 'category_Education_within_2000m', 'category_Medical_within_2000m', 'category_Public_Market_within_1000m', 'pet_policy_binary', 'dataset', 'LGBM_predict', 'XGB_predict', 'CB_predict', 'RF_predict']

First 10 rows:
    

In [39]:
output_df.head()

,district,bedroom_count,property_age,saleable_area,travel_time_to_cbd,walking_time_to_mtr,total_poi_within_1000m,category_Education_within_2000m,category_Medical_within_2000m,category_Public_Market_within_1000m,pet_policy_binary,dataset,LGBM_predict,XGB_predict,CB_predict,RF_predict
0,Ho Man Tin,3,36,978,26,16,838,484,64,3,1,train,RF,RF,RF,RF
1,Happy Valley,3,40,750,21,9,797,164,27,5,1,train,XGB,XGB,RF,RF
2,Tai Wai,1,37,282,45,18,391,115,6,10,1,train,XGB,XGB,CatBoost,CatBoost
3,Sai Ying Pun,3,48,737,14,9,760,145,25,7,1,train,LGBM,LGBM,RF,CatBoost
4,Tsing Yi,2,39,381,41,19,484,112,5,6,1,train,RF,RF,RF,RF


In [40]:
output_df.shape

(6156, 16)

In [44]:
all_consolidated_df.shape

(6156, 20)

In [46]:
all_consolidated_df_indexed = all_consolidated_df.reset_index(drop=False).rename(columns={'index': 'row_id'})
output_df_indexed = output_df.reset_index(drop=False).rename(columns={'index': 'row_id'})

combined_df = pd.merge(
    all_consolidated_df_indexed,
    output_df_indexed[['row_id', 'LGBM_predict', 'XGB_predict', 'CB_predict', 'RF_predict']],
    on='row_id',
    how='inner'
)

combined_df = combined_df.drop(columns=['row_id'])
print(combined_df.shape)

(6156, 24)


In [49]:
def calculate_ensemble_predicted(row):
    lgbm_choice = row['LGBM_predict']
    xgb_choice = row['XGB_predict']
    cb_choice = row['CB_predict']
    rf_choice = row['RF_predict']
    
    votes = pd.Series([lgbm_choice, xgb_choice, cb_choice, rf_choice]).value_counts()
    
    model_to_pred = {
        'LGBM': 'lgbm_predicted',
        'XGB': 'xgb_predicted',
        'CatBoost': 'catboost_predicted',
        'RF': 'rf_predicted'
    }
    
    # If all agree on same model, use only that model
    if len(votes) == 1:
        return row[model_to_pred[votes.index[0]]]
    
    # Otherwise, use weighted ensemble
    top_model = votes.index[0]
    remaining_models = votes.index[1:]
    
    ensemble = 0.5 * row[model_to_pred[top_model]]
    remaining_weight = 0.5 / len(remaining_models)
    
    for model in remaining_models:
        ensemble += remaining_weight * row[model_to_pred[model]]
    
    return ensemble

combined_df['ensemble_predicted'] = combined_df.apply(calculate_ensemble_predicted, axis=1)

In [108]:
combined_df.head()

,district,bedroom_count,property_age,saleable_area,travel_time_to_cbd,walking_time_to_mtr,total_poi_within_1000m,category_Education_within_2000m,category_Medical_within_2000m,category_Public_Market_within_1000m,...,absolute_error,lgbm_predicted,xgb_predicted,catboost_predicted,rf_predicted,LGBM_predict,XGB_predict,CB_predict,RF_predict,ensemble_predicted
0,Kowloon City,3,36,978,26,16,838,484,64,3,...,1.289491e+05,1.237105e+07,12540288.0,1.422219e+07,1.249192e+07,RF,RF,RF,RF,1.249192e+07
1,Wan Chai,3,40,750,21,9,797,164,27,5,...,2.823989e+05,1.328240e+07,12856448.0,1.239582e+07,1.284752e+07,XGB,XGB,RF,RF,1.285198e+07
2,Sha Tin,1,37,282,45,18,391,115,6,10,...,5.018914e+05,3.801891e+06,3411503.0,4.082155e+06,3.662088e+06,XGB,XGB,CatBoost,CatBoost,3.746829e+06
3,Central and Western,3,48,737,14,9,760,145,25,7,...,2.516439e+04,1.065484e+07,10797545.0,1.185482e+07,1.125219e+07,LGBM,LGBM,RF,CatBoost,1.110417e+07
4,Kwai Tsing,2,39,381,41,19,484,112,5,6,...,1.227934e+06,4.127934e+06,4054065.0,4.219507e+06,3.968541e+06,RF,RF,RF,RF,3.968541e+06


   Model       R²  MAPE (%)  MdAPE (%)
    LGBM 0.922421  9.858898   7.229400
     XGB 0.919888 10.167304   7.583526
CatBoost 0.923766  9.959270   7.389122
      RF 0.913071  9.786598   6.582805
Ensemble 0.931690  8.474010   5.712131


In [111]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, median_absolute_error
import numpy as np
import pandas as pd

def calculate_mdape(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

# Filter data to only include the test dataset
test_df = combined_df[combined_df['dataset'] == 'test']

models = {
    'LGBM': 'lgbm_predicted',
    'XGB': 'xgb_predicted',
    'CatBoost': 'catboost_predicted',
    'RF': 'rf_predicted',
    'Ensemble': 'ensemble_predicted'
}

results = []

for model_name, pred_col in models.items():
    r2 = r2_score(test_df['actual_price'], test_df[pred_col])
    mape = mean_absolute_percentage_error(test_df['actual_price'], test_df[pred_col]) * 100
    mdape = calculate_mdape(test_df['actual_price'], test_df[pred_col])
    
    results.append({
        'Model': model_name,
        'R²': r2,
        'MAPE (%)': mape,
        'MdAPE (%)': mdape
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

   Model       R²  MAPE (%)  MdAPE (%)
    LGBM 0.842774 12.368314   8.582569
     XGB 0.845082 12.290859   8.743598
CatBoost 0.845423 12.402399   8.588082
      RF 0.837192 13.735604   9.023464
Ensemble 0.849676 12.287352   8.519365


In [114]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, median_absolute_error
import numpy as np
import pandas as pd

def calculate_mdape(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

# Filter data to only include the test dataset
test_df = combined_df[combined_df['dataset'] == 'test'].copy()

# Create price quartiles
test_df['price_quartile'] = pd.qcut(test_df['actual_price'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

models = {
    'LGBM': 'lgbm_predicted',
    'XGB': 'xgb_predicted',
    'CatBoost': 'catboost_predicted',
    'RF': 'rf_predicted',
    'Ensemble': 'ensemble_predicted'
}

# Overall results
print("OVERALL PERFORMANCE:")
print("=" * 50)
results = []

for model_name, pred_col in models.items():
    r2 = r2_score(test_df['actual_price'], test_df[pred_col])
    mape = mean_absolute_percentage_error(test_df['actual_price'], test_df[pred_col]) * 100
    mdape = calculate_mdape(test_df['actual_price'], test_df[pred_col])
    
    results.append({
        'Model': model_name,
        'R²': r2,
        'MAPE (%)': mape,
        'MdAPE (%)': mdape
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Error distribution across price quartiles
print("\n\nERROR DISTRIBUTION ACROSS PRICE QUARTILES:")
print("=" * 50)

# Show quartile ranges
quartile_ranges = test_df.groupby('price_quartile')['actual_price'].agg(['min', 'max', 'count'])
print("Price Quartile Ranges:")
print(quartile_ranges)

print("\nPerformance by Quartile:")
for quartile in ['Q1', 'Q2', 'Q3', 'Q4']:
    quartile_data = test_df[test_df['price_quartile'] == quartile]
    print(f"\n{quartile} ({len(quartile_data)} properties):")
    
    quartile_results = []
    for model_name, pred_col in models.items():
        r2 = r2_score(quartile_data['actual_price'], quartile_data[pred_col])
        mape = mean_absolute_percentage_error(quartile_data['actual_price'], quartile_data[pred_col]) * 100
        mdape = calculate_mdape(quartile_data['actual_price'], quartile_data[pred_col])
        
        quartile_results.append({
            'Model': model_name,
            'R²': r2,
            'MAPE (%)': mape,
            'MdAPE (%)': mdape
        })
    
    quartile_results_df = pd.DataFrame(quartile_results)
    print(quartile_results_df.to_string(index=False))

OVERALL PERFORMANCE:
   Model       R²  MAPE (%)  MdAPE (%)
    LGBM 0.842774 12.368314   8.582569
     XGB 0.845082 12.290859   8.743598
CatBoost 0.845423 12.402399   8.588082
      RF 0.837192 13.735604   9.023464
Ensemble 0.849676 12.287352   8.519365


ERROR DISTRIBUTION ACROSS PRICE QUARTILES:
Price Quartile Ranges:
                     min       max  count
price_quartile                           
Q1               1680000   5480000    308
Q2               5490000   7500000    317
Q3               7550000  10500000    301
Q4              10600000  48500000    306

Performance by Quartile:

Q1 (308 properties):
   Model        R²  MAPE (%)  MdAPE (%)
    LGBM -0.277309 16.761825  10.136453
     XGB -0.288110 16.597657  10.713573
CatBoost -0.174851 16.440753  10.611932
      RF -0.795768 20.025538  12.379256
Ensemble -0.324315 16.914691  10.650959

Q2 (317 properties):
   Model        R²  MAPE (%)  MdAPE (%)
    LGBM -1.310672  9.856102   7.469549
     XGB -1.062315  9.649193   6.99

In [112]:
# Check for missing values in predictions
print("Missing values in predictions:")
for model_name, pred_col in models.items():
    missing_count = test_df[pred_col].isna().sum()
    print(f"{model_name}: {missing_count} missing values")

# Check if predictions are reasonable
print("\nPrediction statistics:")
for model_name, pred_col in models.items():
    print(f"\n{model_name}:")
    print(f"  Min: {test_df[pred_col].min():,.0f}")
    print(f"  Max: {test_df[pred_col].max():,.0f}")
    print(f"  Mean: {test_df[pred_col].mean():,.0f}")

print(f"\nActual price statistics:")
print(f"  Min: {test_df['actual_price'].min():,.0f}")
print(f"  Max: {test_df['actual_price'].max():,.0f}")
print(f"  Mean: {test_df['actual_price'].mean():,.0f}")

# Check a sample of data
print("\nSample data (first 5 rows):")
sample_cols = ['actual_price'] + list(models.values())
print(test_df[sample_cols].head())

Missing values in predictions:
LGBM: 0 missing values
XGB: 0 missing values
CatBoost: 0 missing values
RF: 0 missing values
Ensemble: 0 missing values

Prediction statistics:

LGBM:
  Min: 2,511,342
  Max: 48,437,836
  Mean: 8,847,929

XGB:
  Min: 2,621,734
  Max: 49,212,504
  Mean: 8,850,445

CatBoost:
  Min: 2,745,827
  Max: 54,185,155
  Mean: 8,884,067

RF:
  Min: 2,799,780
  Max: 45,519,024
  Mean: 8,875,125

Ensemble:
  Min: 2,799,780
  Max: 54,185,155
  Mean: 8,877,571

Actual price statistics:
  Min: 1,680,000
  Max: 48,500,000
  Mean: 8,847,192

Sample data (first 5 rows):
      actual_price  lgbm_predicted  xgb_predicted  catboost_predicted  \
4924       8700000    8.181195e+06      8597520.0        8.867575e+06   
4925      23000000    1.989366e+07     20328726.0        1.916435e+07   
4926       7700000    6.030642e+06      6100254.5        6.194364e+06   
4927       5000000    4.399743e+06      4430923.5        4.510727e+06   
4928       3800000    4.638800e+06      5217821

In [84]:
import numpy as np
import pandas as pd
from scipy import stats

# ============================================================================
# UPDATED WorthinessScorer Class
# ============================================================================

class WorthinessScorer:
    def __init__(self):
        """Initialize the worthiness scorer with updated preference weights and mappings"""
        
        # District scores mapping (age_2 = "25-44", age_4 = "65+")
        self.district_scores = {
            'age_2': {  # 25-44
                'Central and Western': 1.000,
                'Wan Chai': 0.744,
                'Islands': 0.621,
                'Yau Tsim Mong': 0.602,
                'Southern': 0.586,
                'Eastern': 0.526,
                'Sha Tin': 0.396,
                'Kwun Tong': 0.394,
                'Tuen Mun': 0.394,
                'Tai Po': 0.386,
                'North': 0.384,
                'Yuen Long': 0.361,
                'Kowloon City': 0.335,
                'Kwai Tsing': 0.303,
                'Tsuen Wan': 0.286,
                'Sham Shui Po': 0.219,
                'Sai Kung': 0.201,
                'Wong Tai Sin': 0.000
            },
            'age_4': {  # 65+
                'Central and Western': 0.000,
                'Wan Chai': 0.256,
                'Islands': 0.379,
                'Yau Tsim Mong': 0.398,
                'Southern': 0.414,
                'Eastern': 0.474,
                'Sha Tin': 0.604,
                'Kwun Tong': 0.606,
                'Tuen Mun': 0.606,
                'Tai Po': 0.614,
                'North': 0.616,
                'Yuen Long': 0.639,
                'Kowloon City': 0.665,
                'Kwai Tsing': 0.697,
                'Tsuen Wan': 0.714,
                'Sham Shui Po': 0.781,
                'Sai Kung': 0.799,
                'Wong Tai Sin': 1.000
            }
        }
        
        # Walking time to MTR scores
        self.walking_time_scores = {
            'age_2': {  # 25-44
                0: 1.000, 1: 1.000, 2: 1.000, 3: 1.000, 4: 1.000, 5: 1.000,
                6: 1.000, 7: 1.000, 8: 1.000, 9: 1.000, 10: 1.000, 11: 1.000,
                12: 1.000, 13: 0.875, 14: 0.750, 15: 0.625, 16: 0.500,
                17: 0.375, 18: 0.250, 19: 0.125
            },
            'age_4': {  # 65+
                0: 0.800, 1: 0.800, 2: 0.800, 3: 0.800, 4: 0.800, 5: 0.800,
                6: 0.800, 7: 0.800, 8: 0.800, 9: 0.800, 10: 0.800, 11: 0.800,
                12: 0.800, 13: 0.675, 14: 0.550, 15: 0.425, 16: 0.300,
                17: 0.175, 18: 0.050, 19: 0.000
            }
        }
        
        # Bedroom count scores
        self.bedroom_scores = {
            'age_2': {  # 25-44
                'range_1_2': 0.522041,
                'range_3': 0.372007,
                'range_4': 0.890750,
                'range_5_plus': 0.771534
            },
            'age_4': {  # 65+
                'range_1_2': 0.620547,
                'range_3': 0.605019,
                'range_4': 0.044497,
                'range_5_plus': 0.266851
            }
        }
        
        # POI scores (MUF=Public_Market, SCH=Education, HNC=Medical)
        self.poi_scores = {
            'age_2': {  # 25-44
                'MUF': 0.029093,  # Municipal Facility (Public Market)
                'SCH': 0.665867,  # Educational
                'HNC': 1.000     # Health Care (Medical)
            },
            'age_4': {  # 65+
                'MUF': 0.890925,  # Municipal Facility (Public Market)
                'SCH': 0.300066,  # Educational
                'HNC': 0.101520   # Health Care (Medical)
            }
        }
        
        # Component weights for final multiplier
        # worthiness_multiplier = 0.08×bedroom + 0.02×area + 0.25×poi + 0.5×location + 0.15×walking_to_mtr
        self.component_weights = {
            'bedroom': 0.08,
            'area': 0.02,
            'poi': 0.25,
            'location': 0.50,
            'walking_to_mtr': 0.15
        }
        
        # Area benchmarks (keeping from original)
        self.benchmarks = {
            'saleable_area': {
                'median': 522.0,
                'q1': 429.0,
                'q3': 687.0,
                'mean': 558.09
            }
        }
    
    def _get_bedroom_score(self, bedroom_count, age_group):
        """Get bedroom score based on count and age group"""
        if bedroom_count <= 2:
            return self.bedroom_scores[age_group]['range_1_2']
        elif bedroom_count == 3:
            return self.bedroom_scores[age_group]['range_3']
        elif bedroom_count == 4:
            return self.bedroom_scores[age_group]['range_4']
        else:  # 5+
            return self.bedroom_scores[age_group]['range_5_plus']
    
    def _get_district_score(self, district, age_group):
        """Get district score based on district name and age group"""
        return self.district_scores[age_group].get(district, 0.0)
    
    def _get_walking_time_score(self, walking_time, age_group):
        """Get walking time score based on minutes to MTR"""
        walking_time = int(walking_time) if not pd.isna(walking_time) else 20
        
        if walking_time >= 20:
            return 0.0
        else:
            return self.walking_time_scores[age_group].get(walking_time, 0.0)
    
    def _calculate_bedroom_multiplier(self, bedroom_count, age_group):
        """Calculate bedroom preference multiplier"""
        score = self._get_bedroom_score(bedroom_count, age_group)
        # Normalize around 1.0 (using average of all bedroom scores as baseline)
        avg_score = np.mean(list(self.bedroom_scores[age_group].values()))
        multiplier = score / avg_score
        return multiplier
    
    def _calculate_area_multiplier(self, saleable_area, age_group):
        """Calculate area importance multiplier"""
        area_median = self.benchmarks['saleable_area']['median']
        area_ratio = saleable_area / area_median
        
        # Simple area multiplier: larger areas get slight bonus, smaller get slight penalty
        multiplier = 0.8 + (area_ratio * 0.4)  # Range roughly 0.8 to 1.2
        return max(0.5, min(1.5, multiplier))  # Cap between 0.5 and 1.5
    
    def _calculate_poi_multiplier(self, row, age_group):
        """Calculate POI accessibility multiplier"""
        # Get POI counts
        public_market_count = row.get('category_Public_Market_within_1000m', 0)
        education_count = row.get('category_Education_within_2000m', 0)
        medical_count = row.get('category_Medical_within_2000m', 0)
        
        # Get POI scores for age group
        muf_score = self.poi_scores[age_group]['MUF']
        sch_score = self.poi_scores[age_group]['SCH']
        hnc_score = self.poi_scores[age_group]['HNC']
        
        # Normalize POI counts (cap at reasonable levels)
        public_market_norm = min(public_market_count / 5, 1.0)
        education_norm = min(education_count / 10, 1.0)
        medical_norm = min(medical_count / 5, 1.0)
        
        # Calculate weighted POI score
        poi_score = (
            muf_score * public_market_norm +
            sch_score * education_norm +
            hnc_score * medical_norm
        )
        
        # Normalize to multiplier range (0.7 to 1.3)
        max_possible_score = max(muf_score, sch_score, hnc_score)
        normalized_score = poi_score / max_possible_score if max_possible_score > 0 else 0
        multiplier = 0.7 + (normalized_score * 0.6)
        
        return multiplier
    
    def _calculate_location_multiplier(self, district, age_group):
        """Calculate location quality multiplier based on district"""
        district_score = self._get_district_score(district, age_group)
        # Convert district score to multiplier (0.8 to 1.2 range)
        multiplier = 0.8 + (district_score * 0.4)
        return multiplier
    
    def _calculate_walking_multiplier(self, walking_time, age_group):
        """Calculate walking time to MTR multiplier"""
        walking_score = self._get_walking_time_score(walking_time, age_group)
        # Convert walking score to multiplier (0.8 to 1.2 range)
        multiplier = 0.8 + (walking_score * 0.4)
        return multiplier
    
    def calculate_worthiness_score(self, row, age_group):
        """Calculate worthiness score for a property"""
        
        # Use ensemble_predicted as base price
        if 'ensemble_predicted' not in row.index or pd.isna(row['ensemble_predicted']):
            raise ValueError("ensemble_predicted column missing or null")
        
        base_price = row['ensemble_predicted']
        actual_price = row['actual_price']
        
        # Calculate all multipliers
        bedroom_mult = self._calculate_bedroom_multiplier(row['bedroom_count'], age_group)
        area_mult = self._calculate_area_multiplier(row['saleable_area'], age_group)
        poi_mult = self._calculate_poi_multiplier(row, age_group)
        location_mult = self._calculate_location_multiplier(row['district'], age_group)
        walking_mult = self._calculate_walking_multiplier(row['walking_time_to_mtr'], age_group)
        
        # Combined multiplier using new weights
        # worthiness_multiplier = 0.08×bedroom + 0.02×area + 0.25×poi + 0.5×location + 0.15×walking_to_mtr
        combined_multiplier = (
            self.component_weights['bedroom'] * bedroom_mult +
            self.component_weights['area'] * area_mult +
            self.component_weights['poi'] * poi_mult +
            self.component_weights['location'] * location_mult +
            self.component_weights['walking_to_mtr'] * walking_mult
        )
        
        worthiness_price = base_price * combined_multiplier
        
        return {
            'age_group': age_group,
            'actual_price': actual_price,
            'base_price': base_price,
            'worthiness_price': worthiness_price,
            'worthiness_multiplier': combined_multiplier,
            'bedroom_multiplier': bedroom_mult,
            'area_multiplier': area_mult,
            'poi_multiplier': poi_mult,
            'location_multiplier': location_mult,
            'walking_multiplier': walking_mult,
            'price_difference_vs_base': worthiness_price - base_price,
            'price_difference_vs_base_pct': ((worthiness_price / base_price) - 1) * 100,
        }


# ============================================================================
# STEP 2: Define MultiplierAnalyzer Class (unchanged)
# ============================================================================

class MultiplierAnalyzer:
    """Analyze worthiness multiplier distributions before creating scores"""
    
    def __init__(self, scorer):
        self.scorer = scorer
    
    def calculate_all_multipliers(self, df):
        """Calculate worthiness multipliers for all properties in dataframe"""
        results = []
        
        print("Calculating multipliers for all properties...")
        print(f"Total properties: {len(df)}")
        total = len(df)
        errors = 0
        
        for idx, row in df.iterrows():
            try:
                age_2_result = self.scorer.calculate_worthiness_score(row, 'age_2')
                age_4_result = self.scorer.calculate_worthiness_score(row, 'age_4')
                
                results.append({
                    'index': idx,
                    'age_2_multiplier': age_2_result['worthiness_multiplier'],
                    'age_2_worthiness_price': age_2_result['worthiness_price'],
                    'age_2_premium_pct': age_2_result['price_difference_vs_base_pct'],
                    'age_4_multiplier': age_4_result['worthiness_multiplier'],
                    'age_4_worthiness_price': age_4_result['worthiness_price'],
                    'age_4_premium_pct': age_4_result['price_difference_vs_base_pct'],
                    'better_for': 'age_2' if age_2_result['worthiness_price'] > age_4_result['worthiness_price'] else 'age_4',
                    'worthiness_gap': abs(age_2_result['worthiness_price'] - age_4_result['worthiness_price'])
                })
                
                if len(results) % 500 == 0:
                    print(f"  Processed {len(results)}/{total} properties ({len(results)/total*100:.1f}%)...")
                    
            except Exception as e:
                errors += 1
                if errors <= 5:  # Only print first 5 errors
                    print(f"  ⚠️  Error at index {idx}: {str(e)[:50]}")
                continue
        
        if errors > 5:
            print(f"  ⚠️  ... and {errors - 5} more errors (total: {errors} errors)")
        
        print(f"✅ Completed! Processed {len(results)}/{total} properties (success rate: {len(results)/total*100:.1f}%)")
        
        return pd.DataFrame(results)
    
    def print_distribution_analysis(self, multipliers_df):
        """Print detailed distribution analysis for multipliers"""
        
        if len(multipliers_df) == 0:
            print("❌ No valid multipliers calculated!")
            return
        
        print("\n" + "="*100)
        print("WORTHINESS MULTIPLIER DISTRIBUTION ANALYSIS")
        print("="*100)
        
        for age_group in ['age_2', 'age_4']:
            col_name = f'{age_group}_multiplier'
            
            if col_name not in multipliers_df.columns:
                print(f"❌ Column {col_name} not found!")
                continue
                
            multipliers = multipliers_df[col_name].dropna()
            
            if len(multipliers) == 0:
                print(f"❌ No valid multipliers for {age_group}!")
                continue
            
            print(f"\n{'='*100}")
            print(f"AGE GROUP: {age_group.upper()} ({'25-44' if age_group == 'age_2' else '65+'})")
            print(f"{'='*100}")
            
            # Basic statistics
            print(f"\n📊 SUMMARY STATISTICS:")
            print(f"  • Count:        {len(multipliers):>10,}")
            print(f"  • Mean:         {multipliers.mean():>10.4f}")
            print(f"  • Std Dev:      {multipliers.std():>10.4f}")
            print(f"  • Min:          {multipliers.min():>10.4f}")
            print(f"  • Max:          {multipliers.max():>10.4f}")
            print(f"  • Range:        {multipliers.max() - multipliers.min():>10.4f}")
            
            # Percentile analysis
            percentiles = [0, 5, 10, 25, 50, 75, 90, 95, 100]
            values = np.percentile(multipliers, percentiles)
            
            print(f"\n📈 PERCENTILE BREAKDOWN:")
            print(f"  {'Percentile':<15} {'Multiplier':<15} {'Interpretation':<40}")
            print(f"  {'-'*70}")
            
            interpretations = [
                "Worst fit (minimum)",
                "Bottom 5% - Very poor fit",
                "Bottom 10% - Poor fit",
                "Q1 - Below average",
                "Median - Average fit",
                "Q3 - Above average",
                "Top 10% - Very good fit",
                "Top 5% - Excellent fit",
                "Best fit (maximum)"
            ]
            
            for p, v, interp in zip(percentiles, values, interpretations):
                print(f"  {str(p)+'th':<15} {v:<15.4f} {interp:<40}")
            
            # Distribution by ranges
            print(f"\n📊 DISTRIBUTION BY RANGE:")
            ranges = [
                (0, 0.85, "Very Poor (<0.85)"),
                (0.85, 0.90, "Poor (0.85-0.90)"),
                (0.90, 0.95, "Below Average (0.90-0.95)"),
                (0.95, 1.00, "Slightly Below (0.95-1.00)"),
                (1.00, 1.05, "Slightly Above (1.00-1.05)"),
                (1.05, 1.10, "Above Average (1.05-1.10)"),
                (1.10, 1.15, "Good (1.10-1.15)"),
                (1.15, 2.00, "Excellent (>1.15)")
            ]
            
            print(f"  {'Range':<30} {'Count':<10} {'Percentage':<15} {'Cumulative %':<15}")
            print(f"  {'-'*70}")
            
            cumulative = 0
            for low, high, label in ranges:
                count = ((multipliers >= low) & (multipliers < high)).sum()
                pct = (count / len(multipliers)) * 100
                cumulative += pct
                print(f"  {label:<30} {count:<10,} {pct:<14.2f}% {cumulative:<14.2f}%")
        
        # Comparison
        print(f"\n{'='*100}")
        print("COMPARISON: AGE_2 (25-44) vs AGE_4 (65+)")
        print(f"{'='*100}")
        
        age_2_mult = multipliers_df['age_2_multiplier']
        age_4_mult = multipliers_df['age_4_multiplier']
        
        print(f"\n📊 MEAN COMPARISON:")
        print(f"  • age_2 (25-44) mean:   {age_2_mult.mean():.4f}")
        print(f"  • age_4 (65+) mean:     {age_4_mult.mean():.4f}")
        print(f"  • Difference:           {age_2_mult.mean() - age_4_mult.mean():.4f}")
        
        correlation = multipliers_df['age_2_multiplier'].corr(multipliers_df['age_4_multiplier'])
        print(f"\n📊 CORRELATION:")
        print(f"  • Correlation coefficient: {correlation:.4f}")
        
        better_for_counts = multipliers_df['better_for'].value_counts()
        print(f"\n📊 WHICH AGE GROUP VALUES PROPERTIES MORE:")
        for age, count in better_for_counts.items():
            pct = (count / len(multipliers_df)) * 100
            age_label = "25-44" if age == "age_2" else "65+"
            print(f"  • Better for {age} ({age_label}): {count:>6,} properties ({pct:>5.1f}%)")
        
        print(f"\n{'='*100}")
    
    def suggest_score_tiers(self, multipliers_df):
        """Suggest tier boundaries based on actual data distribution"""
        print("\n" + "="*100)
        print("SUGGESTED SCORE TIERS")
        print("="*100)
        
        for age_group in ['age_2', 'age_4']:
            col_name = f'{age_group}_multiplier'
            multipliers = multipliers_df[col_name].dropna()
            age_label = "25-44" if age_group == "age_2" else "65+"
            
            print(f"\n{'─'*100}")
            print(f"AGE GROUP: {age_group.upper()} ({age_label})")
            print(f"{'─'*100}")
            
            p10 = np.percentile(multipliers, 10)
            p40 = np.percentile(multipliers, 40)
            p60 = np.percentile(multipliers, 60)
            p75 = np.percentile(multipliers, 75)
            p90 = np.percentile(multipliers, 90)
            
            print(f"\n💡 RECOMMENDED TIER STRUCTURE:")
            print(f"\n  {'Tier':<20} {'Score Range':<15} {'Multiplier Range':<30}")
            print(f"  {'-'*65}")
            print(f"  {'Poor Fit':<20} {'1-20':<15} {'< {:.4f}':<30}".format(p10))
            print(f"  {'Below Average':<20} {'21-40':<15} {'{:.4f} - {:.4f}':<30}".format(p10, p40))
            print(f"  {'Average':<20} {'41-60':<15} {'{:.4f} - {:.4f}':<30}".format(p40, p60))
            print(f"  {'Above Average':<20} {'61-75':<15} {'{:.4f} - {:.4f}':<30}".format(p60, p75))
            print(f"  {'Good Fit':<20} {'76-90':<15} {'{:.4f} - {:.4f}':<30}".format(p75, p90))
            print(f"  {'Excellent Fit':<20} {'91-100':<15} {'> {:.4f}':<30}".format(p90))
        
        print(f"\n{'='*100}")


# ============================================================================
# STEP 3: Run Analysis Function (unchanged)
# ============================================================================

def run_multiplier_analysis(df, scorer):
    """Run complete multiplier analysis"""
    analyzer = MultiplierAnalyzer(scorer)
    multipliers_df = analyzer.calculate_all_multipliers(df)
    analyzer.print_distribution_analysis(multipliers_df)
    analyzer.suggest_score_tiers(multipliers_df)
    return multipliers_df

In [100]:
combined_df_filtered.columns

Index(['district', 'bedroom_count', 'property_age', 'saleable_area',
       'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m',
       'category_Education_within_2000m', 'category_Medical_within_2000m',
       'category_Public_Market_within_1000m', 'pet_policy_binary', 'dataset',
       'actual_price', 'lgb_predicted_price', 'prediction_error',
       'absolute_error', 'lgbm_predicted', 'xgb_predicted',
       'catboost_predicted', 'rf_predicted', 'LGBM_predict', 'XGB_predict',
       'CB_predict', 'RF_predict', 'ensemble_predicted'],
      dtype='object')

In [92]:
# Get districts used in WorthinessScorer
scorer_districts = [
    'Central and Western',
    'Wan Chai',
    'Islands',
    'Yau Tsim Mong',
    'Southern',
    'Eastern',
    'Sha Tin',
    'Kwun Tong',
    'Tuen Mun',
    'Tai Po',
    'North',
    'Yuen Long',
    'Kowloon City',
    'Kwai Tsing',
    'Tsuen Wan',
    'Sham Shui Po',
    'Sai Kung',
    'Wong Tai Sin'
]

# Filter combined_df to only include scorer districts
combined_df_filtered = combined_df[combined_df['district'].isin(scorer_districts)].copy()

print(f"Original: {len(combined_df):,} properties")
print(f"Filtered: {len(combined_df_filtered):,} properties")

Original: 6,156 properties
Filtered: 267 properties


In [103]:
combined_df_filtered.head()

,district,bedroom_count,property_age,saleable_area,travel_time_to_cbd,walking_time_to_mtr,total_poi_within_1000m,category_Education_within_2000m,category_Medical_within_2000m,category_Public_Market_within_1000m,...,absolute_error,lgbm_predicted,xgb_predicted,catboost_predicted,rf_predicted,LGBM_predict,XGB_predict,CB_predict,RF_predict,ensemble_predicted
0,Kowloon City,3,36,978,26,16,838,484,64,3,...,1.289491e+05,1.237105e+07,12540288.0,1.422219e+07,1.249192e+07,RF,RF,RF,RF,1.249192e+07
1,Wan Chai,3,40,750,21,9,797,164,27,5,...,2.823989e+05,1.328240e+07,12856448.0,1.239582e+07,1.284752e+07,XGB,XGB,RF,RF,1.285198e+07
2,Sha Tin,1,37,282,45,18,391,115,6,10,...,5.018914e+05,3.801891e+06,3411503.0,4.082155e+06,3.662088e+06,XGB,XGB,CatBoost,CatBoost,3.746829e+06
3,Central and Western,3,48,737,14,9,760,145,25,7,...,2.516439e+04,1.065484e+07,10797545.0,1.185482e+07,1.125219e+07,LGBM,LGBM,RF,CatBoost,1.110417e+07
4,Kwai Tsing,2,39,381,41,19,484,112,5,6,...,1.227934e+06,4.127934e+06,4054065.0,4.219507e+06,3.968541e+06,RF,RF,RF,RF,3.968541e+06


In [101]:
scorer = WorthinessScorer()
multipliers_df = run_multiplier_analysis(combined_df_filtered, scorer)

Calculating multipliers for all properties...
Total properties: 6156
  Processed 500/6156 properties (8.1%)...
  Processed 1000/6156 properties (16.2%)...
  Processed 1500/6156 properties (24.4%)...
  Processed 2000/6156 properties (32.5%)...
  Processed 2500/6156 properties (40.6%)...
  Processed 3000/6156 properties (48.7%)...
  Processed 3500/6156 properties (56.9%)...
  Processed 4000/6156 properties (65.0%)...
  Processed 4500/6156 properties (73.1%)...
  Processed 5000/6156 properties (81.2%)...
  Processed 5500/6156 properties (89.3%)...
  Processed 6000/6156 properties (97.5%)...
✅ Completed! Processed 6156/6156 properties (success rate: 100.0%)

WORTHINESS MULTIPLIER DISTRIBUTION ANALYSIS

AGE GROUP: AGE_2 (25-44)

📊 SUMMARY STATISTICS:
  • Count:             6,156
  • Mean:             1.1459
  • Std Dev:          0.0739
  • Min:              0.8130
  • Max:              1.3495
  • Range:            0.5365

📈 PERCENTILE BREAKDOWN:
  Percentile      Multiplier      Interpretat

In [102]:
# Print percentiles for both age groups
print("="*100)
print("DETAILED PERCENTILE ANALYSIS (Every 10th Percentile)")
print("="*100)

for age_group in ['age_2', 'age_4']:
    col_name = f'{age_group}_multiplier'
    multipliers = multipliers_df[col_name].dropna()
    
    print(f"\n{'─'*100}")
    print(f"AGE GROUP: {age_group.upper()}")
    print(f"{'─'*100}")
    
    print(f"\n{'Percentile':<15} {'Multiplier':<15} {'Premium/Discount':<20}")
    print(f"{'-'*50}")
    
    # Calculate percentiles from 0 to 100 in steps of 10
    for percentile in range(0, 101, 10):
        value = np.percentile(multipliers, percentile)
        premium_pct = (value - 1.0) * 100
        
        print(f"{str(percentile)+'th':<15} {value:<15.4f} {premium_pct:>+6.2f}%")

# Print combined (both age groups together)
print(f"\n{'='*100}")
print("COMBINED (BOTH AGE GROUPS)")
print(f"{'='*100}")

all_multipliers = pd.concat([
    multipliers_df['age_2_multiplier'],
    multipliers_df['age_4_multiplier']
])

print(f"\n{'Percentile':<15} {'Multiplier':<15} {'Premium/Discount':<20}")
print(f"{'-'*50}")

for percentile in range(0, 101, 10):
    value = np.percentile(all_multipliers, percentile)
    premium_pct = (value - 1.0) * 100
    
    print(f"{str(percentile)+'th':<15} {value:<15.4f} {premium_pct:>+6.2f}%")

print(f"\n{'='*100}")

DETAILED PERCENTILE ANALYSIS (Every 10th Percentile)

────────────────────────────────────────────────────────────────────────────────────────────────────
AGE GROUP: AGE_2
────────────────────────────────────────────────────────────────────────────────────────────────────

Percentile      Multiplier      Premium/Discount    
--------------------------------------------------
0th             0.8130          -18.70%
10th            1.0749           +7.49%
20th            1.1137          +11.37%
30th            1.1283          +12.83%
40th            1.1393          +13.93%
50th            1.1453          +14.53%
60th            1.1583          +15.83%
70th            1.1699          +16.99%
80th            1.1916          +19.16%
90th            1.2303          +23.03%
100th           1.3495          +34.95%

────────────────────────────────────────────────────────────────────────────────────────────────────
AGE GROUP: AGE_4
───────────────────────────────────────────────────────────────

In [105]:
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore

# ============================================================================
# STEP 1: Calculate Unified Percentile Thresholds (UPDATED)
# ============================================================================

def calculate_unified_thresholds(multipliers_df):
    """
    Calculate unified percentile thresholds from combined age group data
    
    Returns:
        dict: Percentile thresholds for scoring
    """
    # Combine multipliers from both age groups
    all_multipliers = pd.concat([
        multipliers_df['age_2_multiplier'],
        multipliers_df['age_4_multiplier']
    ])
    
    # Use your actual benchmark data
    actual_thresholds = {
        0: 0.8130,
        10: 1.0756,
        20: 1.1106,
        30: 1.1306,
        40: 1.1439,
        50: 1.1595,
        60: 1.1760,
        70: 1.2033,
        80: 1.2310,
        90: 1.2648,
        100: 1.3495
    }
    
    print("\n" + "="*100)
    print("UNIFIED PERCENTILE THRESHOLDS (Combined Age Groups) - ACTUAL DATA")
    print("="*100)
    print(f"\n{'Percentile':<15} {'Multiplier':<15} {'Premium/Discount':<20}")
    print(f"{'-'*50}")
    
    for percentile in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
        value = actual_thresholds[percentile]
        premium_pct = (value - 1.0) * 100
        print(f"{str(percentile)+'th':<15} {value:<15.4f} {premium_pct:>+6.2f}%")
    
    print("="*100)
    
    return all_multipliers, actual_thresholds


# ============================================================================
# STEP 2: Convert Multiplier to Score (1-100) - UPDATED
# ============================================================================

def multiplier_to_score(multiplier, thresholds):
    """
    Convert multiplier to fine-grained score (1-100) based on actual percentile thresholds
    
    Args:
        multiplier: The worthiness multiplier for a property
        thresholds: Dictionary with actual percentile thresholds
    
    Returns:
        int: Score from 1 to 100
    """
    # Handle edge cases
    if multiplier <= thresholds[0]:
        return 1
    if multiplier >= thresholds[100]:
        return 100
    
    # Find which percentile range the multiplier falls into
    percentile_points = sorted(thresholds.keys())
    
    for i in range(len(percentile_points) - 1):
        lower_percentile = percentile_points[i]
        upper_percentile = percentile_points[i + 1]
        
        lower_value = thresholds[lower_percentile]
        upper_value = thresholds[upper_percentile]
        
        if lower_value <= multiplier < upper_value:
            # Linear interpolation between the two percentiles
            ratio = (multiplier - lower_value) / (upper_value - lower_value)
            score = lower_percentile + ratio * (upper_percentile - lower_percentile)
            return max(1, min(100, int(round(score))))
    
    # If we get here, multiplier is exactly at the highest threshold
    return 100


# ============================================================================
# STEP 3: Generate Complete Assessment DataFrame (UPDATED)
# ============================================================================

def generate_age_assessment(combined_df, scorer, multipliers_df, thresholds):
    """
    Generate complete age assessment for all properties
    
    Args:
        combined_df: Original dataframe with all property data
        scorer: WorthinessScorer instance
        multipliers_df: DataFrame with pre-calculated multipliers
        thresholds: Dictionary with actual percentile thresholds
    
    Returns:
        DataFrame: Original columns + age assessment columns
    """
    print("\n" + "="*100)
    print("GENERATING AGE ASSESSMENT FOR ALL PROPERTIES")
    print("="*100)
    
    results = []
    total = len(combined_df)
    errors = 0
    
    print(f"\nProcessing {total} properties...")
    
    for idx, row in combined_df.iterrows():
        try:
            # Calculate worthiness for both age groups
            age_2_result = scorer.calculate_worthiness_score(row, 'age_2')
            age_4_result = scorer.calculate_worthiness_score(row, 'age_4')
            
            # Convert multipliers to scores (1-100) using actual thresholds
            age_2_score = multiplier_to_score(
                age_2_result['worthiness_multiplier'], 
                thresholds
            )
            age_4_score = multiplier_to_score(
                age_4_result['worthiness_multiplier'], 
                thresholds
            )
            
            # Determine which age group values it more
            if age_2_result['worthiness_price'] > age_4_result['worthiness_price']:
                better_for = 'age_2'
            elif age_4_result['worthiness_price'] > age_2_result['worthiness_price']:
                better_for = 'age_4'
            else:
                better_for = 'equal'
            
            # Create result dictionary with all columns
            result = row.to_dict()
            
            # Add age_2 columns
            result['age_2_worthiness_price'] = age_2_result['worthiness_price']
            result['age_2_worthiness_score'] = age_2_score
            result['age_2_worthiness_multiplier'] = age_2_result['worthiness_multiplier']
            
            # Add age_4 columns
            result['age_4_worthiness_price'] = age_4_result['worthiness_price']
            result['age_4_worthiness_score'] = age_4_score
            result['age_4_worthiness_multiplier'] = age_4_result['worthiness_multiplier']
            
            # Add comparison column
            result['better_for'] = better_for
            
            results.append(result)
            
            if len(results) % 500 == 0:
                print(f"  Processed {len(results)}/{total} properties ({len(results)/total*100:.1f}%)...")
                
        except Exception as e:
            errors += 1
            if errors <= 5:
                print(f"  ⚠️  Error at index {idx}: {str(e)[:100]}")
            continue
    
    if errors > 5:
        print(f"  ⚠️  ... and {errors - 5} more errors (total: {errors} errors)")
    
    print(f"\n✅ Completed! Successfully processed {len(results)}/{total} properties")
    
    # Create DataFrame
    assessment_df = pd.DataFrame(results)
    
    return assessment_df


# ============================================================================
# STEP 4: Main Execution Function (FIXED)
# ============================================================================

def create_age_assessment_csv(combined_df, scorer, multipliers_df, output_path):
    """
    Complete pipeline to create age assessment CSV
    
    Args:
        combined_df: Input dataframe with property data
        scorer: WorthinessScorer instance
        multipliers_df: Pre-calculated multipliers (from run_multiplier_analysis)
        output_path: Path to save CSV file
    """
    print("\n" + "="*100)
    print("AGE ASSESSMENT PIPELINE")
    print("="*100)
    
    # Step 1: Calculate unified thresholds using actual data
    all_multipliers, thresholds = calculate_unified_thresholds(multipliers_df)
    
    # Step 2: Generate assessment dataframe
    assessment_df = generate_age_assessment(
        combined_df, 
        scorer, 
        multipliers_df, 
        thresholds  # Pass thresholds instead of all_multipliers
    )
    
    # Step 3: Save to CSV
    print(f"\n💾 Saving to: {output_path}")
    assessment_df.to_csv(output_path, index=False)
    
    # Step 4: Print summary statistics
    print("\n" + "="*100)
    print("SUMMARY STATISTICS")
    print("="*100)
    
    print(f"\n📊 DATAFRAME INFO:")
    print(f"  • Total properties: {len(assessment_df):,}")
    print(f"  • Total columns: {len(assessment_df.columns)}")
    
    print(f"\n📊 NEW COLUMNS ADDED:")
    new_cols = [
        'age_2_worthiness_price',
        'age_2_worthiness_score',
        'age_2_worthiness_multiplier',
        'age_4_worthiness_price',
        'age_4_worthiness_score',
        'age_4_worthiness_multiplier',
        'better_for'
    ]
    for col in new_cols:
        print(f"  • {col}")
    
    print(f"\n📊 SCORE DISTRIBUTION:")
    print(f"\n  AGE_2 Scores:")
    print(f"    • Min:    {assessment_df['age_2_worthiness_score'].min()}")
    print(f"    • Mean:   {assessment_df['age_2_worthiness_score'].mean():.1f}")
    print(f"    • Median: {assessment_df['age_2_worthiness_score'].median():.0f}")
    print(f"    • Max:    {assessment_df['age_2_worthiness_score'].max()}")
    
    print(f"\n  AGE_4 Scores:")
    print(f"    • Min:    {assessment_df['age_4_worthiness_score'].min()}")
    print(f"    • Mean:   {assessment_df['age_4_worthiness_score'].mean():.1f}")
    print(f"    • Median: {assessment_df['age_4_worthiness_score'].median():.0f}")
    print(f"    • Max:    {assessment_df['age_4_worthiness_score'].max()}")
    
    print(f"\n📊 BETTER FOR DISTRIBUTION:")
    better_for_counts = assessment_df['better_for'].value_counts()
    for group, count in better_for_counts.items():
        pct = (count / len(assessment_df)) * 100
        print(f"  • {group:10s}: {count:>6,} properties ({pct:>5.1f}%)")
    
    # FIXED: Correct bin edges and labels
    print(f"\n📊 SCORE TIER DISTRIBUTION (using actual thresholds):")
    
    # Fixed: 6 bin edges for 5 labels
    tier_bins = [0, 20, 40, 60, 80, 100]  # 6 edges
    tier_labels = ['Poor (1-20)', 'Below Avg (21-40)', 'Average (41-60)', 'Above Avg (61-80)', 'Excellent (81-100)']  # 5 labels
    
    print(f"\n  AGE_2 Score Tiers:")
    age_2_tiers = pd.cut(
        assessment_df['age_2_worthiness_score'],
        bins=tier_bins,
        labels=tier_labels
    )
    for tier, count in age_2_tiers.value_counts().sort_index().items():
        pct = (count / len(assessment_df)) * 100
        print(f"  • {tier:25s}: {count:>6,} ({pct:>5.1f}%)")
    
    print(f"\n  AGE_4 Score Tiers:")
    age_4_tiers = pd.cut(
        assessment_df['age_4_worthiness_score'],
        bins=tier_bins,
        labels=tier_labels
    )
    for tier, count in age_4_tiers.value_counts().sort_index().items():
        pct = (count / len(assessment_df)) * 100
        print(f"  • {tier:25s}: {count:>6,} ({pct:>5.1f}%)")
    
    print(f"\n📊 MULTIPLIER vs SCORE VALIDATION:")
    print(f"\n  Sample mappings (first 10 properties):")
    print(f"  {'Age_2 Multiplier':<18} {'Age_2 Score':<12} {'Age_4 Multiplier':<18} {'Age_4 Score':<12}")
    print(f"  {'-'*60}")
    for i in range(min(10, len(assessment_df))):
        print(f"  {assessment_df.iloc[i]['age_2_worthiness_multiplier']:<18.4f} "
              f"{assessment_df.iloc[i]['age_2_worthiness_score']:<12} "
              f"{assessment_df.iloc[i]['age_4_worthiness_multiplier']:<18.4f} "
              f"{assessment_df.iloc[i]['age_4_worthiness_score']:<12}")
    
    print("\n" + "="*100)
    print("✅ AGE ASSESSMENT CSV CREATED SUCCESSFULLY!")
    print("="*100)
    print(f"\nFile saved to: {output_path}")
    print(f"\nSample of first 5 rows:")
    print(assessment_df[['ensemble_predicted', 'actual_price', 
                         'age_2_worthiness_price', 'age_2_worthiness_score',
                         'age_4_worthiness_price', 'age_4_worthiness_score',
                         'better_for']].head())
    
    return assessment_df


# ============================================================================
# READY TO USE!
# ============================================================================

print("✅ Age Assessment Pipeline Ready! (Fixed bin edges)")
print("\nUsage:")
print("  assessment_df = create_age_assessment_csv(")
print("      combined_df=combined_df,")
print("      scorer=scorer,")
print("      multipliers_df=multipliers_df,")
print("      output_path='/Users/clarencemarvin/Downloads/regularized/age_assessment.csv'")
print("  )")

✅ Age Assessment Pipeline Ready! (Fixed bin edges)

Usage:
  assessment_df = create_age_assessment_csv(
      combined_df=combined_df,
      scorer=scorer,
      multipliers_df=multipliers_df,
      output_path='/Users/clarencemarvin/Downloads/regularized/age_assessment.csv'
  )


In [83]:
assessment_df.columns

Index(['district', 'bedroom_count', 'property_age', 'saleable_area',
       'travel_time_to_cbd', 'walking_time_to_mtr', 'total_poi_within_1000m',
       'category_Education_within_2000m', 'category_Medical_within_2000m',
       'category_Public_Market_within_1000m', 'pet_policy_binary', 'dataset',
       'actual_price', 'lgb_predicted_price', 'prediction_error',
       'absolute_error', 'lgbm_predicted', 'xgb_predicted',
       'catboost_predicted', 'rf_predicted', 'LGBM_predict', 'XGB_predict',
       'CB_predict', 'RF_predict', 'ensemble_predicted',
       'age_2_worthiness_price', 'age_2_worthiness_score',
       'age_2_worthiness_multiplier', 'age_4_worthiness_price',
       'age_4_worthiness_score', 'age_4_worthiness_multiplier', 'better_for'],
      dtype='object')

In [90]:
assessment_df = create_age_assessment_csv(
    combined_df=combined_df,
    scorer=scorer,
    multipliers_df=multipliers_df,
    output_path='/Users/clarencemarvin/Downloads/regularized/age_assessment.csv'
)
assessment_df.head()


AGE ASSESSMENT PIPELINE

UNIFIED PERCENTILE THRESHOLDS (Combined Age Groups) - ACTUAL DATA

Percentile      Multiplier      Premium/Discount    
--------------------------------------------------
0th             0.7692          -23.08%
10th            0.9690           -3.10%
20th            1.0223           +2.23%
30th            1.0492           +4.92%
40th            1.0731           +7.31%
50th            1.0829           +8.29%
60th            1.0949           +9.49%
70th            1.0978           +9.78%
80th            1.1107          +11.07%
90th            1.1136          +11.36%
100th           1.3166          +31.66%

GENERATING AGE ASSESSMENT FOR ALL PROPERTIES

Processing 6156 properties...
  Processed 500/6156 properties (8.1%)...
  Processed 1000/6156 properties (16.2%)...
  Processed 1500/6156 properties (24.4%)...
  Processed 2000/6156 properties (32.5%)...
  Processed 2500/6156 properties (40.6%)...
  Processed 3000/6156 properties (48.7%)...
  Processed 3500/6156 pr

,district,bedroom_count,property_age,saleable_area,travel_time_to_cbd,walking_time_to_mtr,total_poi_within_1000m,category_Education_within_2000m,category_Medical_within_2000m,category_Public_Market_within_1000m,...,CB_predict,RF_predict,ensemble_predicted,age_2_worthiness_price,age_2_worthiness_score,age_2_worthiness_multiplier,age_4_worthiness_price,age_4_worthiness_score,age_4_worthiness_multiplier,better_for
0,Ho Man Tin,3,36,978,26,16,838,484,64,3,...,RF,RF,1.249192e+07,1.316730e+07,32,1.054066,1.282399e+07,22,1.026583,age_2
1,Happy Valley,3,40,750,21,9,797,164,27,5,...,RF,RF,1.285198e+07,1.392263e+07,50,1.083306,1.431811e+07,90,1.114078,age_4
2,Tai Wai,1,37,282,45,18,391,115,6,10,...,CatBoost,CatBoost,3.746829e+06,3.933851e+06,30,1.049915,3.990891e+06,37,1.065138,age_4
3,Sai Ying Pun,3,48,737,14,9,760,145,25,7,...,RF,CatBoost,1.110417e+07,1.202700e+07,50,1.083107,1.236870e+07,90,1.113878,age_4
4,Tsing Yi,2,39,381,41,19,484,112,5,6,...,RF,RF,3.968541e+06,4.142887e+06,28,1.043932,4.221161e+06,36,1.063656,age_4


In [91]:
assessment_df.drop(columns=['better_for']).to_csv(
    '/Users/clarencemarvin/Downloads/regularized/age_df.csv', 
    index=False
)
print("✅ Done!")

✅ Done!
